In [1]:
import os, sys
import unidecode
import re
import json
import gc

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Ce notebook analyse les réponses aux questions en cherchant des liens entre les mots les plus utilisés, dans le but de comprendre les demandes les plus récurrentes des francais.

Dans un premier temps, les réponses sont préprocessées et transformées en liste de mots-clés. Puis, pour chaque réponse, úne analyse basique de la proximité des mots est réalisée. Bien que simple, cette methode permet de faire apparaitre des thèmes récurrents et de comprendre la teneur générale de ces demandes.

Les trois métriques utilisées sont :
+ co-occurence des mots : $co-occurence(i, j) = \sum_{users} 1_{i \in user} 1_{j \in user}$
+ confiance : $confiance(i -> j) = P(j | i)$
+ lift : $lift(i, j) = \frac{P(i \cap j)}{P(i)P(j)}$

La co-occurence permet d'évaluer les mots présents de manière commune, et donne une première idée des thèmes généraux.

La confiance donne une idée de la présence de $j$ quand $i$ est déjà présent : il surpondère les mots peu présents par rapport à la co-occurence.

Le lift estime le surplus de présence commune de $i$ et $j$ par rapport au cas où $i$ et $j$ sont indépendants. Cela permet de trouver des relations de causalité plus intéressantes, notamment dans des cas où certains mots sont peu présents mais très liés.

Pour chaque question :
+ On récupère les mots les plus employés sur ce thème
+ Pour chacun de ces mots, on regarde pour chaque métrique les mots les plus liés, ce qui donne une bonne idée des liens et des demandes

On peut ensuite raffiner l'analyse sur des mots moins présents mais porteurs de sens.

In [2]:
# function definitions

def compute_similarities(question_name, df_res):
    qu = df_res[question_name].dropna()
    df_qu_one_hot = compute_one_hot(qu)
    df_cov = compute_covariance(df_qu_one_hot)
    df_confidence = compute_confidence(df_cov)
    df_lift = compute_lift(df_cov, df_qu_one_hot)
    
    return qu, df_qu_one_hot, df_cov, df_confidence, df_lift
    
def compute_one_hot(qu):
    qu_words = []
    for i in qu:
        qu_words.extend(i)
    qu_words = list(set(qu_words))
    dic_qu_words = {word:i for i, word in enumerate(qu_words)}
    dic_qu_words_inverse = {i:word for i, word in enumerate(qu_words)}
    
    qu_one_hot = np.zeros((len(qu), len(qu_words)), dtype=bool)
    for i, li_word in enumerate(qu):
        for word in li_word:
            qu_one_hot[i, dic_qu_words[word]] = True
            
    df_qu_one_hot = pd.DataFrame(columns=qu_words, data=qu_one_hot).to_sparse(fill_value=False)

    
    temp_mask = np.sum(df_qu_one_hot.values, axis=0, dtype=np.uint16) > 250
    words_enough = df_qu_one_hot.columns.values[temp_mask]
    df_qu_one_hot = df_qu_one_hot[words_enough].to_sparse(fill_value=0)
    
    del temp_mask
    del qu_one_hot
    gc.collect()
    
    return df_qu_one_hot

def compute_covariance(df_qu_one_hot):
    N = df_qu_one_hot.shape[1]
    
    cov = np.zeros((N, N), dtype=np.uint16)
    np.matmul(df_qu_one_hot.values.T, df_qu_one_hot.values, out=cov)
    
    df_cov = pd.DataFrame(columns=df_qu_one_hot.columns, index=df_qu_one_hot.columns, data=cov) * (cov > 100)
    del cov
    gc.collect()
    
    return df_cov

def compute_confidence(df_cov):
    df_confidence = np.diag(1 / np.diag(df_cov)).dot(df_cov).astype(np.float32)
    df_confidence = pd.DataFrame(index=df_cov.index, columns=df_cov.index, data=df_confidence)
    return df_confidence

def compute_lift(df_cov, df_qu_one_hot):
    inv_diag_cov = 1 / np.diag(df_cov.values)
    lift = inv_diag_cov.reshape((-1, 1)) * df_cov.values * inv_diag_cov.reshape((1, -1))
    df_lift = pd.DataFrame(columns=df_qu_one_hot.columns,
                             index=df_qu_one_hot.columns,
                             data=lift).replace(0.0, np.nan).astype(np.float32)
    del lift
    del inv_diag_cov
    gc.collect()
    
    return df_lift

# Loading of files and preprocessing

In [3]:
PATH_TO_FILE = "./proposals_cleaned/democratie.json"

In [4]:
with open(PATH_TO_FILE, "r") as f:
    raw_json = [s.strip() for s in f.readlines()]
    
parsed_json = [json.loads(raw_json[i]) for i in range(len(raw_json))]
del raw_json

In [5]:
col = ["titre"]
for ques in parsed_json[0]["responses"]:
    col.append(ques["question"]["id"]) 
        
data = []
for elt in parsed_json:
    resp = {"titre" : elt["titre"]}
    for ques in elt["responses"]:
        if ques["question"]["title"] == "Si oui, comment ?":
            if ques["question"]["id"] == "121":
                ques["question"]["title"] += "_RIP"
            else:
                ques["question"]["title"] += "_Senat"
        try:
            text = json.loads((ques["value"]))
            resp[ques["question"]["title"]] = ''
            if text["labels"] != []:
                resp[ques["question"]["title"]] += ", ".join(text["labels"])
            if text["other"] is not None:
                resp[ques["question"]["title"]] += ", " + text["other"]
            if resp[ques["question"]["title"]] == '':
                resp[ques["question"]["title"]] = np.nan
        except:
            if ques["value"] is not None and ques["value"][0] != "{":
                resp[ques["question"]["title"]] = ques["value"]
            else:
                resp[ques["question"]["title"]] = np.nan
    data.append(resp)

In [6]:
res = pd.DataFrame.from_dict(data)
del data
gc.collect()

0

In [7]:
res.notna().mean(axis=0).sort_values(ascending=False)

titre                                                                                                                                                                                                             1.000000
Le non-cumul des mandats instauré en 2017 pour les parlementaires (députés et sénateurs) est :                                                                                                                    0.859542
Faut-il prendre en compte le vote blanc ?                                                                                                                                                                         0.853150
En dehors des élus politiques, faut-il donner un rôle plus important aux associations et aux organisations syndicales et professionnelles ?                                                                       0.835577
Faut-il faciliter le déclenchement du référendum d'initiative partagée (le RIP est organisé à l'initiative de membres du Par

# Find important words

In [8]:
li_word_count = []
for elt in res.values:
    for item in elt:
        if type(item) == str:
            li_word_count.extend([unidecode.unidecode(x.lower()) for x in re.findall(r'\w+', item)])

In [9]:
dic_word_count = {}
for i in li_word_count:
    if len(i) > 0:
        if i in dic_word_count.keys():
            dic_word_count[i] +=1
        else:
            dic_word_count[i] = 1

In [10]:
df_word_count = pd.DataFrame.from_dict(dic_word_count, orient="index")
df_word_count.sort_values(0, ascending=False)
df_word_count.columns = ["freq_in_debate"]

In [11]:
del li_word_count
del dic_word_count

gc.collect()

202

In [12]:
df_word_count = df_word_count[df_word_count["freq_in_debate"] > 100]

In [13]:
li_stopwords = [unidecode.unidecode(x.strip().lower()) for x in open("french_stopwords.txt", 'r').readlines()]
li_stopwords = set(li_stopwords)
li_stopwords.remove("aucun")
li_stopwords.remove("tout")
li_stopwords.remove("tous")
li_stopwords.remove("non")

In [14]:
new_data = []
for i, elt in enumerate(res.index):
    temp_res = []
    for col in res.columns:
        item = res.loc[elt, col]
        if type(item) == str:
            li_item = [unidecode.unidecode(x.lower()) for x in re.findall(r'\w+', item) if unidecode.unidecode(x.lower()) not in li_stopwords]
            temp_res.append(li_item)
        else:
            temp_res.append(None)
    new_data.append(temp_res)

In [15]:
new_res = pd.DataFrame(new_data, columns=res.columns).to_sparse()
del new_data
gc.collect()

0

# Question 1 :  En qui faites-vous le plus confiance pour vous faire représenter dans la société et pourquoi ?

In [16]:
question_1 = "En qui faites-vous le plus confiance pour vous faire représenter dans la société et pourquoi ?"
qu_1,df_qu_1_one_hot,df_cov_1,df_confidence_1,df_lift_1 = compute_similarities(question_1,new_res)

In [17]:
for i in df_qu_1_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_1[i].sort_values(ascending=False).head(10))

elus


elus             7797
locaux           1284
confiance        1119
citoyens          891
deputes           843
tous              659
maire             653
maires            588
politiques        587
representants     571
Name: elus, dtype: uint16

maire


maire        7340
depute       1393
proximite     772
commune       736
citoyens      706
elus          653
elu           546
president     484
terrain       412
connait       399
Name: maire, dtype: uint16

confiance


...

proximite    1341
maire         772
elus          384
maires        197
elu           171
depute        166
citoyens      144
deputes       106
terrain       106
tout            0
Name: proximite, dtype: uint16

peuple


peuple           1288
elus              452
confiance         227
democratie        185
representants     178
deputes           176
pouvoir           174
citoyens          166
faire             161
politique         155
Name: peuple, dtype: uint16

tout


tout          1250
elus           413
confiance      306
faire          268
politique      239
tous           221
monde          211
citoyens       209
pouvoir        205
politiques     202
Name: tout, dtype: uint16

In [18]:
for i in df_qu_1_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_1[i].sort_values(ascending=False).head(10))

elus


elus                1.000000
locaux              0.882474
nationaux           0.765625
democratiquement    0.687003
suffrage            0.559356
municipaux          0.558065
representent        0.553360
universel           0.527187
elections           0.519588
democratique        0.519231
Name: elus, dtype: float32

maire


maire          1.000000
accessible     0.822006
commune        0.765073
connait        0.726776
administres    0.673611
habitants      0.588679
proximite      0.575690
concitoyens    0.520000
quotidien      0.492239
contact        0.491176
Name: maire, dtype: float32

confiance


...

proximite    1.000000
maire        0.105177
terrain      0.098148
maires       0.083439
elu          0.081662
depute       0.056501
citoyens     0.050883
elus         0.049250
deputes      0.039820
france       0.000000
Name: proximite, dtype: float32

peuple


peuple           1.000000
francais         0.173077
faut             0.165882
democratie       0.155332
representants    0.142514
pouvoir          0.140210
representer      0.126607
non              0.117886
faire            0.106552
tout             0.096800
Name: peuple, dtype: float32

tout


tout        1.000000
cas         0.292339
monde       0.280959
temps       0.280591
pays        0.266019
france      0.255357
place       0.250597
faut        0.224706
systeme     0.196429
francais    0.193681
Name: tout, dtype: float32

In [19]:
for i in df_qu_1_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_1[i].sort_values(ascending=False).head(10))

elus


elus                0.000128
locaux              0.000113
nationaux           0.000098
democratiquement    0.000088
suffrage            0.000072
municipaux          0.000072
representent        0.000071
universel           0.000068
elections           0.000067
democratique        0.000067
Name: elus, dtype: float32

maire


maire          0.000136
accessible     0.000112
commune        0.000104
connait        0.000099
administres    0.000092
habitants      0.000080
proximite      0.000078
concitoyens    0.000071
quotidien      0.000067
contact        0.000067
Name: maire, dtype: float32

confiance


...

proximite    0.000746
maire        0.000078
terrain      0.000073
maires       0.000062
elu          0.000061
depute       0.000042
citoyens     0.000038
elus         0.000037
deputes      0.000030
habitants         NaN
Name: proximite, dtype: float32

peuple


peuple           0.000776
francais         0.000134
faut             0.000129
democratie       0.000121
representants    0.000111
pouvoir          0.000109
representer      0.000098
non              0.000092
faire            0.000083
tout             0.000075
Name: peuple, dtype: float32

tout


tout        0.000800
cas         0.000234
monde       0.000225
temps       0.000224
pays        0.000213
france      0.000204
place       0.000200
faut        0.000180
systeme     0.000157
francais    0.000155
Name: tout, dtype: float32

In [20]:
gc.collect()

147

# Question 2 : En dehors des élus politiques, faut-il donner un rôle plus important aux associations et aux organisations syndicales et professionnelles ?

In [21]:
question_2 = "En dehors des élus politiques, faut-il donner un rôle plus important aux associations et aux organisations syndicales et professionnelles ?"
qu_2,df_qu_2_one_hot,df_cov_2,df_confidence_2,df_lift_2 = compute_similarities(question_2,new_res)

In [22]:
for i in df_qu_2_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_2[i].sort_values(ascending=False).head(10))

oui


oui    19384
non        0
Name: oui, dtype: uint16

non


non    13948
oui        0
Name: non, dtype: uint16

In [23]:
for i in df_qu_2_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_2[i].sort_values(ascending=False).head(10))

oui


oui    1.0
non    0.0
Name: oui, dtype: float32

non


non    1.0
oui    0.0
Name: non, dtype: float32

In [24]:
for i in df_qu_2_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_2[i].sort_values(ascending=False).head(10))

oui


oui    0.000052
non         NaN
Name: oui, dtype: float32

non


non    0.000072
oui         NaN
Name: non, dtype: float32

In [25]:
gc.collect()

14

# Question 3 :  Si oui, à quel type d'associations ou d'organisations ? Et avec quel rôle ?

In [26]:
question_3 = "Si oui, à quel type d'associations ou d'organisations ? Et avec quel rôle ?"
qu_3,df_qu_3_one_hot,df_cov_3,df_confidence_3,df_lift_3 = compute_similarities(question_3, new_res)

In [27]:
for i in df_qu_3_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_3[i].sort_values(ascending=False).head(10))

associations


associations        8786
syndicats           2549
organisations       1603
role                1456
citoyens            1021
non                  956
syndicales           953
consommateurs        651
faire                648
professionnelles     570
Name: associations, dtype: uint16

syndicats


syndicats         5467
associations      2549
role               791
non                588
salaries           570
organisations      486
faut               446
representatifs     435
citoyens           429
pouvoir            400
Name: syndicats, dtype: uint16

organisations


...

politiques       853
associations     474
syndicats        312
organisations    222
role             174
non              170
syndicales       149
citoyens         119
elus             102
faut             102
Name: politiques, dtype: uint16

consommateurs


consommateurs    839
associations     651
defense          253
syndicats        235
role             136
association      125
organisations    119
citoyens         114
environnement    108
connaissent        0
Name: consommateurs, dtype: uint16

defense


defense          789
associations     521
consommateurs    253
environnement    224
syndicats        186
droits           161
organisations    144
role             133
citoyens         128
association      104
Name: defense, dtype: uint16

In [28]:
for i in df_qu_3_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_3[i].sort_values(ascending=False).head(10))

associations


associations     1.000000
caritatives      0.871854
reconnues        0.853147
humanitaires     0.828479
locales          0.798805
consommateurs    0.775924
utilite          0.757895
lucratif         0.751613
but              0.740484
citoyennes       0.739224
Name: associations, dtype: float32

syndicats


syndicats         1.000000
representatifs    0.864811
professionnels    0.651214
salaries          0.633333
travailleurs      0.618012
adhesion          0.582031
representent      0.565674
entreprises       0.552672
obligatoire       0.549632
dialogue          0.521595
Name: syndicats, dtype: float32

organisations


...

politiques       1.000000
elus             0.113082
non              0.096099
faut             0.085427
syndicales       0.072260
organisations    0.067069
role             0.064207
citoyens         0.060683
syndicats        0.057070
associations     0.053949
Name: politiques, dtype: float32

consommateurs


consommateurs    1.000000
defense          0.320659
environnement    0.173355
association      0.083333
associations     0.074095
citoyens         0.058134
role             0.050185
syndicats        0.042985
organisations    0.035952
propositions     0.000000
Name: consommateurs, dtype: float32

defense


defense          1.000000
environnement    0.359551
droits           0.356984
consommateurs    0.301549
association      0.069333
citoyens         0.065273
associations     0.059299
role             0.049077
organisations    0.043505
syndicats        0.034022
Name: defense, dtype: float32

In [29]:
for i in df_qu_3_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_3[i].sort_values(ascending=False).head(10))

associations


associations     0.000114
caritatives      0.000099
reconnues        0.000097
humanitaires     0.000094
locales          0.000091
consommateurs    0.000088
utilite          0.000086
lucratif         0.000086
but              0.000084
citoyennes       0.000084
Name: associations, dtype: float32

syndicats


syndicats         0.000183
representatifs    0.000158
professionnels    0.000119
salaries          0.000116
travailleurs      0.000113
adhesion          0.000106
representent      0.000103
entreprises       0.000101
obligatoire       0.000101
dialogue          0.000095
Name: syndicats, dtype: float32

organisations


...

politiques       0.001172
elus             0.000133
non              0.000113
faut             0.000100
syndicales       0.000085
organisations    0.000079
role             0.000075
citoyens         0.000071
syndicats        0.000067
associations     0.000063
Name: politiques, dtype: float32

consommateurs


consommateurs    0.001192
defense          0.000382
environnement    0.000207
association      0.000099
associations     0.000088
citoyens         0.000069
role             0.000060
syndicats        0.000051
organisations    0.000043
caritatives           NaN
Name: consommateurs, dtype: float32

defense


defense          0.001267
environnement    0.000456
droits           0.000452
consommateurs    0.000382
association      0.000088
citoyens         0.000083
associations     0.000075
role             0.000062
organisations    0.000055
syndicats        0.000043
Name: defense, dtype: float32

In [30]:
gc.collect()

147

# Question 4 :  Que faudrait-il faire pour renouer le lien entre les citoyens et les élus qui les représentent ?

In [31]:
question_4 = "Que faudrait-il faire pour renouer le lien entre les citoyens et les élus qui les représentent ?"
qu_4,df_qu_4_one_hot,df_cov_4,df_confidence_4,df_lift_4 = compute_similarities(question_4, new_res)

In [32]:
for i in df_qu_4_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_4[i].sort_values(ascending=False).head(10))

elus


elus         11882
citoyens      3704
faire         1627
tous          1495
non           1447
politique     1282
vie           1266
tout          1215
faut          1188
elu           1152
Name: elus, dtype: uint16

citoyens


citoyens     7054
elus         3704
faire        1231
tous          932
non           851
politique     846
faut          811
tout          742
elu           738
vie           714
Name: citoyens, dtype: uint16

faire


...

mandats      1652
elus          833
cumul         602
limiter       465
non           439
nombre        423
2             384
politique     362
mandat        348
citoyens      326
Name: mandats, dtype: uint16

politiques


politiques    1649
elus           685
citoyens       549
politique      393
faire          344
tous           325
non            314
tout           270
vie            268
faut           255
Name: politiques, dtype: uint16

assemblee


assemblee    1634
nationale     752
elus          743
citoyens      561
deputes       504
senat         398
faire         319
vote          295
tous          289
non           271
Name: assemblee, dtype: uint16

In [33]:
for i in df_qu_4_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_4[i].sort_values(ascending=False).head(10))

elus


elus            1.000000
exemplaires     0.818182
respectent      0.802920
rendent         0.748503
representent    0.745938
ecoutent        0.743494
devraient       0.715385
fassent         0.703791
frais           0.684669
nationaux       0.682741
Name: elus, dtype: float32

citoyens


citoyens        1.000000
tires           0.819106
sort            0.636455
participer      0.627219
puissent        0.553444
pourraient      0.544803
democratique    0.507463
consulter       0.501567
tirage          0.490741
avis            0.490272
Name: citoyens, dtype: float32

faire


...

mandats       1.000000
cumul         0.690367
limiter       0.622490
maximum       0.461818
duree         0.411243
2             0.359551
interdire     0.327628
nombre        0.295391
metier        0.258794
indemnites    0.253968
Name: mandats, dtype: float32

politiques


politiques    1.000000
hommes        0.701439
partis        0.455036
france        0.162311
politique     0.148246
ans           0.144041
pays          0.141367
francais      0.141158
cas           0.130392
place         0.124598
Name: politiques, dtype: float32

assemblee


assemblee          1.000000
nationale          0.743818
senat              0.547455
presence           0.307914
tires              0.274390
presents           0.269634
sort               0.236657
proportionnelle    0.229860
3                  0.218891
citoyenne          0.217391
Name: assemblee, dtype: float32

In [34]:
for i in df_qu_4_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_4[i].sort_values(ascending=False).head(10))

elus


elus            0.000084
exemplaires     0.000069
respectent      0.000068
rendent         0.000063
representent    0.000063
ecoutent        0.000063
devraient       0.000060
fassent         0.000059
frais           0.000058
nationaux       0.000057
Name: elus, dtype: float32

citoyens


citoyens        0.000142
tires           0.000116
sort            0.000090
participer      0.000089
puissent        0.000078
pourraient      0.000077
democratique    0.000072
consulter       0.000071
tirage          0.000070
avis            0.000070
Name: citoyens, dtype: float32

faire


...

mandats       0.000605
cumul         0.000418
limiter       0.000377
maximum       0.000280
duree         0.000249
2             0.000218
interdire     0.000198
nombre        0.000179
metier        0.000157
indemnites    0.000154
Name: mandats, dtype: float32

politiques


politiques    0.000606
hommes        0.000425
partis        0.000276
france        0.000098
politique     0.000090
ans           0.000087
pays          0.000086
francais      0.000086
cas           0.000079
place         0.000076
Name: politiques, dtype: float32

assemblee


assemblee          0.000612
nationale          0.000455
senat              0.000335
presence           0.000188
tires              0.000168
presents           0.000165
sort               0.000145
proportionnelle    0.000141
3                  0.000134
citoyenne          0.000133
Name: assemblee, dtype: float32

In [35]:
gc.collect()

105

# Question 5 :   Le non-cumul des mandats instauré en 2017 pour les parlementaires (députés et sénateurs) est :

In [36]:
question_5 = "Le non-cumul des mandats instauré en 2017 pour les parlementaires (députés et sénateurs) est :"
qu_5,df_qu_5_one_hot,df_cov_5,df_confidence_5,df_lift_5 = compute_similarities(question_5, new_res)

In [37]:
for i in df_qu_5_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_5[i].sort_values(ascending=False).head(10))

chose


chose       30760
bonne       25881
mauvaise     4879
sais            0
Name: chose, dtype: uint16

bonne


bonne       25881
chose       25881
sais            0
mauvaise        0
Name: bonne, dtype: uint16

mauvaise


chose       4879
mauvaise    4879
sais           0
bonne          0
Name: mauvaise, dtype: uint16

sais


sais        3528
bonne          0
chose          0
mauvaise       0
Name: sais, dtype: uint16

In [38]:
for i in df_qu_5_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_5[i].sort_values(ascending=False).head(10))

chose


bonne       1.0
chose       1.0
mauvaise    1.0
sais        0.0
Name: chose, dtype: float32

bonne


bonne       1.000000
chose       0.841385
sais        0.000000
mauvaise    0.000000
Name: bonne, dtype: float32

mauvaise


mauvaise    1.000000
chose       0.158615
sais        0.000000
bonne       0.000000
Name: mauvaise, dtype: float32

sais


sais        1.0
bonne       0.0
chose       0.0
mauvaise    0.0
Name: sais, dtype: float32

In [39]:
for i in df_qu_5_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_5[i].sort_values(ascending=False).head(10))

chose


bonne       0.000033
chose       0.000033
mauvaise    0.000033
sais             NaN
Name: chose, dtype: float32

bonne


bonne       0.000039
chose       0.000033
mauvaise         NaN
sais             NaN
Name: bonne, dtype: float32

mauvaise


mauvaise    0.000205
chose       0.000033
bonne            NaN
sais             NaN
Name: mauvaise, dtype: float32

sais


sais        0.000283
mauvaise         NaN
chose            NaN
bonne            NaN
Name: sais, dtype: float32

In [40]:
gc.collect()

182

# Question 6 :   Pourquoi ?

In [41]:
question_6 = "Pourquoi ?"
qu_6,df_qu_6_one_hot,df_cov_6,df_confidence_6,df_lift_6 = compute_similarities(question_6, new_res)

In [42]:
for i in df_qu_6_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_6[i].sort_values(ascending=False).head(10))

mandat


mandat     4531
temps      1023
elu         974
cumul       967
mandats     824
elus        803
local       729
depute      629
maire       605
faut        585
Name: mandat, dtype: uint16

cumul


cumul      4512
mandats    1971
non        1573
mandat      967
elus        909
maire       858
temps       789
depute      774
faut        683
elu         614
Name: cumul, dtype: uint16

mandats


...

tout       1549
faire       478
temps       370
cumul       346
mandats     327
mandat      299
elus        274
faut        246
elu         238
depute      233
Name: tout, dtype: uint16

consacrer


consacrer     1535
mandat         541
temps          487
elu            395
pleinement     343
tache          289
elus           268
mandats        209
mission        199
cumul          175
Name: consacrer, dtype: uint16

chose


chose      1461
bonne       912
cumul       397
mandats     297
faire       289
maire       268
depute      264
elus        249
temps       238
non         236
Name: chose, dtype: uint16

In [43]:
for i in df_qu_6_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_6[i].sort_values(ascending=False).head(10))

mandat


mandat        1.000000
local         0.521459
national      0.509044
1             0.398876
municipal     0.393502
exercer       0.391858
conseiller    0.367978
consacrer     0.352443
pleinement    0.334591
elu           0.314194
Name: mandat, dtype: float32

cumul


cumul            1.000000
remunerations    0.660714
non              0.634786
indemnites       0.576531
retraites        0.571970
remuneration     0.516216
probleme         0.496894
mandats          0.451753
salaires         0.444005
revenus          0.405316
Name: cumul, dtype: float32

mandats


...

tout       1.000000
monde      0.384615
faire      0.141253
cumuler    0.094253
temps      0.086469
faut       0.083988
elu        0.076774
cumul      0.076684
travail    0.076520
mandats    0.074948
Name: tout, dtype: float32

consacrer


consacrer     1.000000
pleinement    0.431447
tache         0.329909
mission       0.204312
elu           0.127419
mandat        0.119400
fonction      0.116230
temps         0.113812
mieux         0.111328
pouvoir       0.106250
Name: consacrer, dtype: float32

chose


chose       1.000000
bonne       0.785530
mauvaise    0.750929
non         0.095238
maire       0.092350
deputes     0.092145
cumul       0.087988
faire       0.085402
depute      0.084833
terrain     0.077110
Name: chose, dtype: float32

In [44]:
for i in df_qu_6_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_6[i].sort_values(ascending=False).head(10))

mandat


mandat        0.000221
local         0.000115
national      0.000112
1             0.000088
municipal     0.000087
exercer       0.000086
conseiller    0.000081
consacrer     0.000078
pleinement    0.000074
elu           0.000069
Name: mandat, dtype: float32

cumul


cumul            0.000222
remunerations    0.000146
non              0.000141
indemnites       0.000128
retraites        0.000127
remuneration     0.000114
probleme         0.000110
mandats          0.000100
salaires         0.000098
revenus          0.000090
Name: cumul, dtype: float32

mandats


...

tout       0.000646
monde      0.000248
faire      0.000091
cumuler    0.000061
temps      0.000056
faut       0.000054
elu        0.000050
cumul      0.000050
travail    0.000049
mandats    0.000048
Name: tout, dtype: float32

consacrer


consacrer     0.000651
pleinement    0.000281
tache         0.000215
mission       0.000133
elu           0.000083
mandat        0.000078
fonction      0.000076
temps         0.000074
mieux         0.000073
pouvoir       0.000069
Name: consacrer, dtype: float32

chose


chose       0.000684
bonne       0.000538
mauvaise    0.000514
non         0.000065
maire       0.000063
deputes     0.000063
cumul       0.000060
faire       0.000058
depute      0.000058
terrain     0.000053
Name: chose, dtype: float32

In [45]:
gc.collect()

147

# Question 7 :  Que faudrait-il faire pour mieux représenter les différentes sensibilités politiques ?

In [46]:
question_7 = "Que faudrait-il faire pour mieux représenter les différentes sensibilités politiques ?"
qu_7,df_qu_7_one_hot,df_cov_7,df_confidence_7,df_lift_7 = compute_similarities(question_7, new_res)

In [47]:
for i in df_qu_7_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_7[i].sort_values(ascending=False).head(10))

proportionnelle


proportionnelle    15443
dose                4092
elections           2297
integrale           1425
assemblee           1376
instaurer           1301
introduire          1244
legislatives        1242
vote                1040
majorite             917
Name: proportionnelle, dtype: uint16

dose


dose               4528
proportionnelle    4092
elections           734
introduire          685
legislatives        543
instaurer           447
assemblee           436
petite              402
majorite            378
republique          292
Name: dose, dtype: uint16

elections


...

faire              1342
proportionnelle     447
politiques          277
politique           268
citoyens            209
partis              209
elus                195
faut                185
sensibilites        182
elections           177
Name: faire, dtype: uint16

tous


tous               1297
proportionnelle     552
partis              282
politiques          264
politique           208
elections           187
elus                179
citoyens            174
assemblee           167
vote                159
Name: tous, dtype: uint16

proportionnel


proportionnel       1239
scrutin              268
elections            175
dose                 173
vote                 164
assemblee            102
voix                   0
voter                  0
gauche                 0
proportionnalite       0
Name: proportionnel, dtype: uint16

In [48]:
for i in df_qu_7_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_7[i].sort_values(ascending=False).head(10))

proportionnelle


proportionnelle    1.000000
integrale          0.936884
dose               0.903710
petite             0.887097
introduire         0.856749
25                 0.845272
totale             0.837209
15                 0.834842
prime              0.832817
20                 0.815661
Name: proportionnelle, dtype: float32

dose


dose            1.000000
petite          0.810484
introduire      0.471763
bonne           0.411911
15              0.411765
20              0.401305
stabilite       0.392523
25              0.363897
legislatives    0.353516
30              0.311047
Name: dose, dtype: float32

elections


...

faire        1.000000
politique    0.148724
pouvoir      0.144550
non          0.140133
citoyens     0.139987
tout         0.137931
pays         0.136250
faut         0.134058
parti        0.132753
elus         0.131579
Name: faire, dtype: float32

tous


tous           1.000000
ans            0.340845
representes    0.285714
francais       0.184932
partis         0.155286
elus           0.120783
citoyens       0.116544
parti          0.116431
non            0.116301
politiques     0.115688
Name: tous, dtype: float32

proportionnel


proportionnel       1.000000
scrutin             0.269618
vote                0.071867
elections           0.053305
assemblee           0.050545
dose                0.038207
debat               0.000000
referendum          0.000000
gauche              0.000000
proportionnalite    0.000000
Name: proportionnel, dtype: float32

In [49]:
for i in df_qu_7_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_7[i].sort_values(ascending=False).head(10))

proportionnelle


proportionnelle    0.000065
integrale          0.000061
dose               0.000059
petite             0.000057
introduire         0.000055
25                 0.000055
totale             0.000054
15                 0.000054
prime              0.000054
20                 0.000053
Name: proportionnelle, dtype: float32

dose


dose            0.000221
petite          0.000179
introduire      0.000104
bonne           0.000091
15              0.000091
20              0.000089
stabilite       0.000087
25              0.000080
legislatives    0.000078
30              0.000069
Name: dose, dtype: float32

elections


...

faire        0.000745
politique    0.000111
pouvoir      0.000108
non          0.000104
citoyens     0.000104
tout         0.000103
pays         0.000102
faut         0.000100
parti        0.000099
elus         0.000098
Name: faire, dtype: float32

tous


tous           0.000771
ans            0.000263
representes    0.000220
francais       0.000143
partis         0.000120
elus           0.000093
citoyens       0.000090
parti          0.000090
non            0.000090
politiques     0.000089
Name: tous, dtype: float32

proportionnel


proportionnel     0.000807
scrutin           0.000218
vote              0.000058
elections         0.000043
assemblee         0.000041
dose              0.000031
citoyens               NaN
presidentielle         NaN
senat                  NaN
20                     NaN
Name: proportionnel, dtype: float32

In [50]:
gc.collect()

147

# Question 8 : Pensez-vous qu'il serait souhaitable de réduire le nombre d'élus (hors députés et sénateurs) ?

In [51]:
question_8 = "Pensez-vous qu'il serait souhaitable de réduire le nombre d'élus (hors députés et sénateurs) ?"
qu_8,df_qu_8_one_hot,df_cov_8,df_confidence_8,df_lift_8 = compute_similarities(question_8, new_res)

In [52]:
for i in df_qu_8_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_8[i].sort_values(ascending=False).head(10))

oui


oui    24164
non        0
Name: oui, dtype: uint16

non


non    8770
oui       0
Name: non, dtype: uint16

In [53]:
for i in df_qu_8_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_8[i].sort_values(ascending=False).head(10))

oui


oui    1.0
non    0.0
Name: oui, dtype: float32

non


non    1.0
oui    0.0
Name: non, dtype: float32

In [54]:
for i in df_qu_8_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_8[i].sort_values(ascending=False).head(10))

oui


oui    0.000041
non         NaN
Name: oui, dtype: float32

non


non    0.000114
oui         NaN
Name: non, dtype: float32

In [55]:
gc.collect()

14

# Question 9 :  Si oui, lesquels ?

In [56]:
question_9 = "Si oui, lesquels ?"
qu_9,df_qu_9_one_hot,df_cov_9,df_confidence_9,df_lift_9 = compute_similarities(question_9, new_res)

In [57]:
for i in df_qu_9_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_9[i].sort_values(ascending=False).head(10))

deputes


deputes        5683
senateurs      4352
nombre         1504
reduire        1072
elus            941
senat           862
faut            666
supprimer       653
communes        480
conseillers     476
Name: deputes, dtype: uint16

senateurs


senateurs      5591
deputes        4352
nombre         1209
reduire         937
elus            866
faut            624
supprimer       467
conseillers     453
tous            450
communes        432
Name: senateurs, dtype: uint16

elus


...

conseils          1392
departementaux     694
regionaux          582
communes           355
generaux           332
supprimer          331
elus               325
nombre             265
municipaux         240
regions            189
Name: conseils, dtype: uint16

maires


maires            1382
communes           689
elus               331
nombre             324
conseillers        306
deputes            287
senateurs          258
petites            211
reduire            203
departementaux     202
Name: maires, dtype: uint16

conseil


conseil          1327
departemental     403
regional          356
supprimer         302
elus              288
economique        280
communes          270
social            259
general           253
deputes           231
Name: conseil, dtype: uint16

In [58]:
for i in df_qu_9_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_9[i].sort_values(ascending=False).head(10))

deputes


deputes      1.000000
justement    0.867232
senateurs    0.778394
europeens    0.639456
100          0.614286
moitie       0.574018
question     0.538941
3            0.505643
1            0.492832
senateur     0.444700
Name: deputes, dtype: float32

senateurs


senateurs    1.000000
justement    0.881356
deputes      0.765793
100          0.528571
question     0.526480
moitie       0.456193
1            0.408602
3            0.401806
oui          0.371739
francais     0.370748
Name: senateurs, dtype: float32

elus


...

conseils          1.000000
generaux          0.345833
departementaux    0.247592
municipaux        0.233690
regionaux         0.208080
suppression       0.141361
communautes       0.127132
supprimer         0.119538
communes          0.091260
conseil           0.088922
Name: conseils, dtype: float32

maires


maires         1.000000
petites        0.303597
habitants      0.220913
presidents     0.193487
communes       0.177121
municipaux     0.141188
communautes    0.120155
conseillers    0.101257
commune        0.100577
nombre         0.085398
Name: maires, dtype: float32

conseil


conseil          1.000000
economique       0.838323
social           0.830128
general          0.664042
departemental    0.503121
regional         0.478495
communaute       0.155844
suppression      0.123037
commune          0.122836
supprimer        0.109065
Name: conseil, dtype: float32

In [59]:
for i in df_qu_9_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_9[i].sort_values(ascending=False).head(10))

deputes


deputes      0.000176
justement    0.000153
senateurs    0.000137
europeens    0.000113
100          0.000108
moitie       0.000101
question     0.000095
3            0.000089
1            0.000087
senateur     0.000078
Name: deputes, dtype: float32

senateurs


senateurs    0.000179
justement    0.000158
deputes      0.000137
100          0.000095
question     0.000094
moitie       0.000082
1            0.000073
3            0.000072
oui          0.000066
francais     0.000066
Name: senateurs, dtype: float32

elus


...

conseils          0.000718
generaux          0.000248
departementaux    0.000178
municipaux        0.000168
regionaux         0.000149
suppression       0.000102
communautes       0.000091
supprimer         0.000086
communes          0.000066
conseil           0.000064
Name: conseils, dtype: float32

maires


maires         0.000724
petites        0.000220
habitants      0.000160
presidents     0.000140
communes       0.000128
municipaux     0.000102
communautes    0.000087
conseillers    0.000073
commune        0.000073
nombre         0.000062
Name: maires, dtype: float32

conseil


conseil          0.000754
economique       0.000632
social           0.000626
general          0.000500
departemental    0.000379
regional         0.000361
communaute       0.000117
suppression      0.000093
commune          0.000093
supprimer        0.000082
Name: conseil, dtype: float32

In [60]:
gc.collect()

147

# Question 10 :  Que pensez-vous de la participation des citoyens aux élections et comment les inciter à y participer davantage ?

In [61]:
question_10 = "Que pensez-vous de la participation des citoyens aux élections et comment les inciter à y participer davantage ?"
qu_10,df_qu_10_one_hot,df_cov_10,df_confidence_10,df_lift_10 = compute_similarities(question_10, new_res)

In [62]:
for i in df_qu_10_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_10[i].sort_values(ascending=False).head(10))

vote


vote             15408
obligatoire       9752
blanc             4461
compte            2404
voter             1837
citoyens          1750
faut              1675
participation     1540
elections         1488
non               1037
Name: vote, dtype: uint16

obligatoire


obligatoire      10505
vote              9752
blanc             1957
compte            1125
participation     1038
faut               983
voter              924
elections          850
citoyens           803
amende             779
Name: obligatoire, dtype: uint16

citoyens


...

prendre        1479
compte         1048
vote           1013
blanc           832
obligatoire     447
citoyens        290
faut            252
votes           224
voter           216
elections       203
Name: prendre, dtype: uint16

tous


tous             1449
vote              647
citoyens          498
obligatoire       339
voter             321
faut              304
elus              296
elections         287
politique         257
participation     246
Name: tous, dtype: uint16

citoyen


citoyen          1414
vote              760
obligatoire       450
voter             357
citoyens          289
faut              282
elections         262
devoir            260
participation     252
politique         230
Name: citoyen, dtype: uint16

In [63]:
for i in df_qu_10_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_10[i].sort_values(ascending=False).head(10))

vote


vote              1.000000
electronique      0.972376
blanc             0.963083
reconnaissance    0.950185
obligatoire       0.928320
reconnaitre       0.925170
sanction          0.875000
belgique          0.863874
amende            0.823869
prise             0.822951
Name: vote, dtype: float32

obligatoire


obligatoire    1.000000
belgique       0.824607
rendant        0.812500
sanction       0.777174
amende         0.749759
peine          0.717228
vote           0.632918
semaine        0.573123
droits         0.519139
cas            0.510204
Name: obligatoire, dtype: float32

citoyens


...

prendre      1.000000
compte       0.329663
blanc        0.179620
blancs       0.170662
votes        0.137255
election     0.083578
faire        0.075834
faut         0.074031
vote         0.065745
elections    0.059566
Name: prendre, dtype: float32

tous


tous         1.000000
ans          0.280066
francais     0.150823
elu          0.133983
vie          0.133903
tout         0.124725
candidats    0.123939
pouvoir      0.122144
exemple      0.120047
pays         0.116505
Name: tous, dtype: float32

citoyen


citoyen       1.000000
devoir        0.276008
droits        0.246411
droit         0.153318
tout          0.150404
elu           0.138855
vie           0.132004
democratie    0.120513
pays          0.112190
pouvoir       0.101933
Name: citoyen, dtype: float32

In [64]:
for i in df_qu_10_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_10[i].sort_values(ascending=False).head(10))

vote


vote              0.000065
electronique      0.000063
blanc             0.000063
reconnaissance    0.000062
obligatoire       0.000060
reconnaitre       0.000060
sanction          0.000057
belgique          0.000056
amende            0.000053
prise             0.000053
Name: vote, dtype: float32

obligatoire


obligatoire    0.000095
belgique       0.000078
rendant        0.000077
sanction       0.000074
amende         0.000071
peine          0.000068
vote           0.000060
semaine        0.000055
droits         0.000049
cas            0.000049
Name: obligatoire, dtype: float32

citoyens


...

prendre      0.000676
compte       0.000223
blanc        0.000121
blancs       0.000115
votes        0.000093
election     0.000057
faire        0.000051
faut         0.000050
vote         0.000044
elections    0.000040
Name: prendre, dtype: float32

tous


tous         0.000690
ans          0.000193
francais     0.000104
elu          0.000092
vie          0.000092
tout         0.000086
candidats    0.000086
pouvoir      0.000084
exemple      0.000083
pays         0.000080
Name: tous, dtype: float32

citoyen


citoyen       0.000707
devoir        0.000195
droits        0.000174
droit         0.000108
tout          0.000106
elu           0.000098
vie           0.000093
democratie    0.000085
pays          0.000079
pouvoir       0.000072
Name: citoyen, dtype: float32

In [65]:
gc.collect()

147

# Question 11 :  Faut-il prendre en compte le vote blanc ?

In [66]:
question_11 = "Faut-il prendre en compte le vote blanc ?"
qu_11,df_qu_11_one_hot,df_cov_11,df_confidence_11,df_lift_11 = compute_similarities(question_11, new_res)

In [67]:
for i in df_qu_11_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_11[i].sort_values(ascending=False).head(10))

oui


oui    28021
non        0
Name: oui, dtype: uint16

non


non    6012
oui       0
Name: non, dtype: uint16

In [68]:
for i in df_qu_11_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_11[i].sort_values(ascending=False).head(10))

oui


oui    1.0
non    0.0
Name: oui, dtype: float32

non


non    1.0
oui    0.0
Name: non, dtype: float32

In [69]:
for i in df_qu_11_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_11[i].sort_values(ascending=False).head(10))

oui


oui    0.000036
non         NaN
Name: oui, dtype: float32

non


non    0.000166
oui         NaN
Name: non, dtype: float32

In [70]:
gc.collect()

14

# Question 12 : Si oui, de quelle manière ?

In [71]:
question_12 = "Si oui, de quelle manière ?"
qu_12,df_qu_12_one_hot,df_cov_12,df_confidence_12,df_lift_12 = compute_similarities(question_12, new_res)

In [72]:
for i in df_qu_12_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_12[i].sort_values(ascending=False).head(10))

vote


vote           11815
blanc           8851
election        3412
candidats       2473
compte          1808
votes           1753
majoritaire     1464
candidat        1454
elections       1420
blancs          1299
Name: vote, dtype: uint16

blanc


blanc          9948
vote           8851
election       3295
candidats      2371
compte         1621
votes          1565
majoritaire    1417
elections      1403
candidat       1334
faut           1151
Name: blanc, dtype: uint16

election


...

tour         1555
vote          945
blanc         890
election      591
candidats     529
second        457
votes         409
candidat      376
blancs        363
compte        325
Name: tour, dtype: uint16

suffrages


suffrages    1437
exprimes     1185
vote          601
blanc         524
blancs        400
votes         371
election      356
compte        278
50            263
elu           241
Name: suffrages, dtype: uint16

exprime


exprime      1432
vote          996
blanc         724
suffrage      585
election      242
compte        235
candidats     220
votes         193
candidat      170
majorite      143
Name: exprime, dtype: uint16

In [73]:
for i in df_qu_12_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_12[i].sort_values(ascending=False).head(10))

vote


vote           1.000000
obligatoire    0.966637
blanc          0.889727
considere      0.831461
emporte        0.822981
majoritaire    0.803072
signifie       0.793103
arrive         0.790409
citoyen        0.767386
annule         0.752860
Name: vote, dtype: float32

blanc


blanc          1.000000
emporte        0.922360
signifie       0.782759
majoritaire    0.777290
arrive         0.769094
considere      0.767790
bulletin       0.758242
vote           0.749132
tete           0.710375
citoyen        0.707434
Name: blanc, dtype: float32

election


...

tour              1.000000
second            0.877159
1er               0.784946
presidentielle    0.436782
2                 0.393548
tete              0.243516
arrive            0.236234
exemple           0.224658
elu               0.163848
cas               0.163362
Name: tour, dtype: float32

suffrages


suffrages    1.000000
exprimes     0.515666
elu          0.127378
50           0.118736
scrutin      0.117584
tour         0.109968
nombre       0.106368
blancs       0.104384
majorite     0.092287
compte       0.091357
Name: suffrages, dtype: float32

exprime


exprime         1.000000
suffrage        0.768725
comptabilise    0.148344
compter         0.131227
vote            0.084300
compte          0.077226
elu             0.073467
blanc           0.072778
candidat        0.071369
tour            0.068810
Name: exprime, dtype: float32

In [74]:
for i in df_qu_12_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_12[i].sort_values(ascending=False).head(10))

vote


vote           0.000085
obligatoire    0.000082
blanc          0.000075
considere      0.000070
emporte        0.000070
majoritaire    0.000068
signifie       0.000067
arrive         0.000067
citoyen        0.000065
annule         0.000064
Name: vote, dtype: float32

blanc


blanc          0.000101
emporte        0.000093
signifie       0.000079
majoritaire    0.000078
arrive         0.000077
considere      0.000077
bulletin       0.000076
vote           0.000075
tete           0.000071
citoyen        0.000071
Name: blanc, dtype: float32

election


...

tour              0.000643
second            0.000564
1er               0.000505
presidentielle    0.000281
2                 0.000253
tete              0.000157
arrive            0.000152
exemple           0.000144
elu               0.000105
cas               0.000105
Name: tour, dtype: float32

suffrages


suffrages    0.000696
exprimes     0.000359
elu          0.000089
50           0.000083
scrutin      0.000082
tour         0.000077
nombre       0.000074
blancs       0.000073
majorite     0.000064
compte       0.000064
Name: suffrages, dtype: float32

exprime


exprime         0.000698
suffrage        0.000537
comptabilise    0.000104
compter         0.000092
vote            0.000059
compte          0.000054
elu             0.000051
blanc           0.000051
candidat        0.000050
tour            0.000048
Name: exprime, dtype: float32

In [75]:
gc.collect()

147

# Question 13 : Que faudrait-il faire aujourd'hui pour mieux associer les citoyens aux grandes orientations et à la décision publique ? Comment mettre en place une démocratie plus participative ?

In [76]:
question_13 = "Que faudrait-il faire aujourd'hui pour mieux associer les citoyens aux grandes orientations et à la décision publique ? Comment mettre en place une démocratie plus participative ?"
qu_13,df_qu_13_one_hot,df_cov_13,df_confidence_13,df_lift_13 = compute_similarities(question_13, new_res)

In [77]:
for i in df_qu_13_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_13[i].sort_values(ascending=False).head(10))

referendum


referendum    7479
initiative    1541
citoyens      1341
citoyenne     1013
sujets         870
faire          806
questions      788
place          775
vote           670
decisions      617
Name: referendum, dtype: uint16

citoyens


citoyens      6859
sort          1574
referendum    1341
elus          1240
faire         1231
tires          949
decisions      904
place          811
tous           777
vote           751
Name: citoyens, dtype: uint16

faire


...

niveau        1896
local          908
citoyens       627
national       564
referendum     472
faire          357
elus           279
decisions      274
democratie     271
place          253
Name: niveau, dtype: uint16

debats


debats        1852
citoyens       658
organiser      393
faire          313
elus           283
referendum     264
grands         242
sujets         208
niveau         207
locaux         202
Name: debats, dtype: uint16

mettre


mettre         1827
place          1425
citoyens        574
referendum      520
faire           280
initiative      278
ric             253
faut            235
elus            232
referendums     228
Name: mettre, dtype: uint16

In [78]:
for i in df_qu_13_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_13[i].sort_values(ascending=False).head(10))

referendum


referendum      1.000000
initiative      0.794740
recours         0.772853
populaire       0.743553
signatures      0.662745
citoyenne       0.633125
constitution    0.575540
resultat        0.564477
soumettre       0.468310
importantes     0.459610
Name: referendum, dtype: float32

citoyens


citoyens      1.000000
tires         0.850358
associer      0.696466
tirer         0.685259
sort          0.668082
puissent      0.627820
pourraient    0.592105
participer    0.584541
ensemble      0.546713
senat         0.539062
Name: citoyens, dtype: float32

faire


...

niveau           1.000000
local            0.618108
regional         0.560091
national         0.444444
commune          0.335341
communes         0.312812
region           0.291566
participative    0.167273
participation    0.145861
projets          0.145518
Name: niveau, dtype: float32

debats


debats        1.000000
publics       0.431373
organiser     0.300919
grands        0.238189
locaux        0.169891
publiques     0.149199
reunions      0.134091
participer    0.128502
politiques    0.109290
niveau        0.109177
Name: debats, dtype: float32

mettre


mettre          1.000000
place           0.564133
oeuvre          0.446565
systeme         0.186813
initiative      0.143373
faudrait        0.141711
citoyenne       0.141250
faut            0.121573
propositions    0.119266
nombre          0.118605
Name: mettre, dtype: float32

In [79]:
for i in df_qu_13_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_13[i].sort_values(ascending=False).head(10))

referendum


referendum      0.000134
initiative      0.000106
recours         0.000103
populaire       0.000099
signatures      0.000089
citoyenne       0.000085
constitution    0.000077
resultat        0.000075
soumettre       0.000063
importantes     0.000061
Name: referendum, dtype: float32

citoyens


citoyens      0.000146
tires         0.000124
associer      0.000102
tirer         0.000100
sort          0.000097
puissent      0.000092
pourraient    0.000086
participer    0.000085
ensemble      0.000080
senat         0.000079
Name: citoyens, dtype: float32

faire


...

niveau           0.000527
local            0.000326
regional         0.000295
national         0.000234
commune          0.000177
communes         0.000165
region           0.000154
participative    0.000088
participation    0.000077
projets          0.000077
Name: niveau, dtype: float32

debats


debats        0.000540
publics       0.000233
organiser     0.000162
grands        0.000129
locaux        0.000092
publiques     0.000081
reunions      0.000072
participer    0.000069
politiques    0.000059
niveau        0.000059
Name: debats, dtype: float32

mettre


mettre          0.000547
place           0.000309
oeuvre          0.000244
systeme         0.000102
initiative      0.000078
faudrait        0.000078
citoyenne       0.000077
faut            0.000067
propositions    0.000065
nombre          0.000065
Name: mettre, dtype: float32

In [80]:
gc.collect()

147

# Question 14 : Faut-il faciliter le déclenchement du référendum d'initiative partagée (le RIP est organisé à l'initiative de membres du Parlement soutenu par une partie du corps électoral) qui est applicable depuis 2015 ?

In [81]:
question_14 = "Faut-il faciliter le déclenchement du référendum d'initiative partagée (le RIP est organisé à l'initiative de membres du Parlement soutenu par une partie du corps électoral) qui est applicable depuis 2015 ?"
qu_14,df_qu_14_one_hot,df_cov_14,df_confidence_14,df_lift_14 = compute_similarities(question_14, new_res)

In [82]:
for i in df_qu_14_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_14[i].sort_values(ascending=False).head(10))

oui


oui     17588
sais        0
non         0
Name: oui, dtype: uint16

non


non     8904
sais       0
oui        0
Name: non, dtype: uint16

sais


sais    6835
oui        0
non        0
Name: sais, dtype: uint16

In [83]:
for i in df_qu_14_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_14[i].sort_values(ascending=False).head(10))

oui


oui     1.0
sais    0.0
non     0.0
Name: oui, dtype: float32

non


non     1.0
sais    0.0
oui     0.0
Name: non, dtype: float32

sais


sais    1.0
oui     0.0
non     0.0
Name: sais, dtype: float32

In [84]:
for i in df_qu_14_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_14[i].sort_values(ascending=False).head(10))

oui


oui     0.000057
non          NaN
sais         NaN
Name: oui, dtype: float32

non


non     0.000112
oui          NaN
sais         NaN
Name: non, dtype: float32

sais


sais    0.000146
non          NaN
oui          NaN
Name: sais, dtype: float32

In [85]:
gc.collect()

21

# Question 15 : Si oui, comment ? (RIP)

In [86]:
question_15 = "Si oui, comment ?_RIP"
qu_15,df_qu_15_one_hot,df_cov_15,df_confidence_15,df_lift_15 = compute_similarities(question_15, new_res)

In [88]:
for i in df_qu_15_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_15[i].sort_values(ascending=False).head(10))

referendum


referendum    3288
initiative     887
citoyens       772
nombre         557
faut           509
non            416
citoyenne      400
question       393
questions      349
vote           331
Name: referendum, dtype: uint16

citoyens


citoyens          2871
nombre             839
referendum         772
initiative         621
rip                473
elus               447
parlementaires     433
faut               391
ric                304
parlement          263
Name: citoyens, dtype: uint16

nombre


...

questions     950
referendum    349
citoyens      179
non           157
question      138
oui           133
faut          128
rip           126
voir            0
prendre         0
Name: questions, dtype: uint16

exemple


exemple       858
referendum    318
citoyens      238
nombre        194
initiative    143
rip           133
sujets        125
faut          121
tous          108
signatures    102
Name: exemple, dtype: uint16

tout


tout          812
referendum    299
citoyens      167
faut          150
ric           146
rip           136
faire         125
non           113
nombre        112
initiative    109
Name: tout, dtype: uint16

In [89]:
for i in df_qu_15_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_15[i].sort_values(ascending=False).head(10))

referendum


referendum      1.000000
citoyenne       0.693241
populaire       0.676737
president       0.535714
national        0.520231
initiative      0.505701
constitution    0.473373
choix           0.439216
gouvernement    0.437824
proposition     0.428571
Name: referendum, dtype: float32

citoyens


citoyens          1.000000
parlementaires    0.394353
elus              0.386010
possibilite       0.377698
millions          0.376106
declenche         0.366559
demande           0.355828
initiative        0.354048
avis              0.350993
pouvoir           0.350904
Name: citoyens, dtype: float32

nombre


...

questions     1.000000
oui           0.169211
question      0.142122
non           0.136522
referendum    0.106144
faut          0.082902
rip           0.063032
citoyens      0.062348
voir          0.000000
mise          0.000000
Name: questions, dtype: float32

exemple


exemple       1.000000
tous          0.153846
1             0.136546
sujets        0.117813
referendum    0.096715
signatures    0.094708
citoyens      0.082898
initiative    0.081528
nombre        0.079770
faut          0.078368
Name: exemple, dtype: float32

tout


tout          1.000000
faire         0.123031
non           0.098261
faut          0.097150
sujets        0.097078
ric           0.091307
referendum    0.090937
rip           0.068034
initiative    0.062144
citoyens      0.058168
Name: tout, dtype: float32

In [90]:
for i in df_qu_15_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_15[i].sort_values(ascending=False).head(10))

referendum


referendum      0.000304
citoyenne       0.000211
populaire       0.000206
president       0.000163
national        0.000158
initiative      0.000154
constitution    0.000144
choix           0.000134
gouvernement    0.000133
proposition     0.000130
Name: referendum, dtype: float32

citoyens


citoyens          0.000348
parlementaires    0.000137
elus              0.000134
possibilite       0.000132
millions          0.000131
declenche         0.000128
demande           0.000124
initiative        0.000123
avis              0.000122
pouvoir           0.000122
Name: citoyens, dtype: float32

nombre


...

questions     0.001053
oui           0.000178
question      0.000150
non           0.000144
referendum    0.000112
faut          0.000087
rip           0.000066
citoyens      0.000066
sujets             NaN
seuil              NaN
Name: questions, dtype: float32

exemple


exemple       0.001166
tous          0.000179
1             0.000159
sujets        0.000137
referendum    0.000113
signatures    0.000110
citoyens      0.000097
initiative    0.000095
nombre        0.000093
faut          0.000091
Name: exemple, dtype: float32

tout


tout          0.001232
faire         0.000152
non           0.000121
faut          0.000120
sujets        0.000120
ric           0.000112
referendum    0.000112
rip           0.000084
initiative    0.000077
citoyens      0.000072
Name: tout, dtype: float32

In [91]:
gc.collect()

420

# Question 16 : Que faudrait-il faire pour consulter plus directement les citoyens sur l'utilisation de l'argent public, par l'Etat et les collectivités ?

In [92]:
question_16 = "Que faudrait-il faire pour consulter plus directement les citoyens sur l'utilisation de l'argent public, par l'Etat et les collectivités ?"
qu_16,df_qu_16_one_hot,df_cov_16,df_confidence_16,df_lift_16 = compute_similarities(question_16, new_res)

In [93]:
for i in df_qu_16_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_16[i].sort_values(ascending=False).head(10))

citoyens


citoyens       5015
faire           800
argent          788
elus            677
public          671
consulter       617
depenses        615
comptes         595
utilisation     554
sort            547
Name: citoyens, dtype: uint16

referendum


referendum    2977
citoyens       348
local          285
initiative     271
faire          226
citoyenne      208
internet       177
depenses       164
budget         154
vote           154
Name: referendum, dtype: uint16

faire


...

consulter      1377
citoyens        617
argent          295
faut            258
public          240
utilisation     192
depenses        181
informer        176
elus            176
faire           172
Name: consulter, dtype: uint16

tout


tout        1377
citoyens     370
argent       271
faire        264
public       225
monde        206
depenses     198
elus         196
faut         181
citoyen      176
Name: tout, dtype: uint16

consultation


consultation    1330
citoyens         329
internet         276
faire            157
referendum       131
argent           127
public           115
ligne            111
niveau           109
depenses         106
Name: consultation, dtype: uint16

In [94]:
for i in df_qu_16_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_16[i].sort_values(ascending=False).head(10))

citoyens


citoyens         1.000000
tires            0.864608
sort             0.695044
pourraient       0.538194
participer       0.498534
exprimer         0.498069
consulter        0.448076
representants    0.431677
assemblees       0.428571
permettre        0.413534
Name: citoyens, dtype: float32

referendum


referendum    1.000000
initiative    0.687817
citoyenne     0.416000
local         0.329861
national      0.230000
questions     0.206395
grands        0.198113
locaux        0.168994
choix         0.149051
projets       0.145612
Name: referendum, dtype: float32

faire


...

consulter      1.000000
faut           0.183892
informer       0.157565
faudrait       0.138335
citoyens       0.123031
argent         0.113856
public         0.113422
utilisation    0.109652
tout           0.103849
citoyen        0.087733
Name: consulter, dtype: float32

tout


tout         1.000000
monde        0.795367
citoyen      0.136646
faut         0.129009
pouvoir      0.118268
impots       0.114656
public       0.106333
argent       0.104593
consulter    0.103849
exemple      0.102362
Name: tout, dtype: float32

consultation


consultation    1.000000
ligne           0.122652
internet        0.107351
niveau          0.089198
tout            0.074074
citoyens        0.065603
faire           0.055733
public          0.054348
elus            0.051741
argent          0.049016
Name: consultation, dtype: float32

In [95]:
for i in df_qu_16_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_16[i].sort_values(ascending=False).head(10))

citoyens


citoyens         0.000199
tires            0.000172
sort             0.000139
pourraient       0.000107
participer       0.000099
exprimer         0.000099
consulter        0.000089
representants    0.000086
assemblees       0.000085
permettre        0.000082
Name: citoyens, dtype: float32

referendum


referendum    0.000336
initiative    0.000231
citoyenne     0.000140
local         0.000111
national      0.000077
questions     0.000069
grands        0.000067
locaux        0.000057
choix         0.000050
projets       0.000049
Name: referendum, dtype: float32

faire


...

consulter      0.000726
faut           0.000134
informer       0.000114
faudrait       0.000100
citoyens       0.000089
argent         0.000083
public         0.000082
utilisation    0.000080
tout           0.000075
citoyen        0.000064
Name: consulter, dtype: float32

tout


tout         0.000726
monde        0.000578
citoyen      0.000099
faut         0.000094
pouvoir      0.000086
impots       0.000083
public       0.000077
argent       0.000076
consulter    0.000075
exemple      0.000074
Name: tout, dtype: float32

consultation


consultation    0.000752
ligne           0.000092
internet        0.000081
niveau          0.000067
tout            0.000056
citoyens        0.000049
faire           0.000042
public          0.000041
elus            0.000039
argent          0.000037
Name: consultation, dtype: float32

In [96]:
gc.collect()

140

# Question 17 :  Quel rôle nos assemblées, dont le Sénat et le Conseil économique, social et environnemental, doivent-elles jouer pour représenter nos territoires et la société civile ?

In [97]:
question_17 = "Quel rôle nos assemblées, dont le Sénat et le Conseil économique, social et environnemental, doivent-elles jouer pour représenter nos territoires et la société civile ?"
qu_17,df_qu_17_one_hot,df_cov_17,df_confidence_17,df_lift_17 = compute_similarities(question_17, new_res)

In [98]:
for i in df_qu_17_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_17[i].sort_values(ascending=False).head(10))

senat


senat          5568
cese           2139
assemblee      1499
conseil        1448
role           1324
economique     1217
social          987
supprimer       967
nationale       847
territoires     795
Name: senat, dtype: uint16

role


role          4156
senat         1324
cese           811
conseil        662
assemblee      560
citoyens       529
economique     514
social         439
pouvoir        402
assemblees     387
Name: role, dtype: uint16

cese


...

societe        1204
civile          849
senat           445
territoires     417
cese            346
role            283
representer     259
assemblee       222
citoyens        217
elus            194
Name: societe, dtype: uint16

nationale


nationale      1155
assemblee      1034
senat           847
cese            403
role            333
conseil         268
economique      218
territoires     213
lois            208
pouvoir         204
Name: nationale, dtype: uint16

sert


sert          1127
senat          623
cese           490
conseil        372
economique     337
social         282
supprimer      280
assemblee      203
role           191
faut           140
Name: sert, dtype: uint16

In [99]:
for i in df_qu_17_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_17[i].sort_values(ascending=False).head(10))

senat


senat            1.000000
chambre          0.761488
nationale        0.733333
represente       0.706827
assemblee        0.699160
supprime         0.690789
mode             0.687023
inutile          0.664706
collectivites    0.657692
suppression      0.648094
Name: senat, dtype: float32

role


role              1.000000
jouer             0.880196
actuel            0.669841
consultatif       0.661058
controle          0.471729
actuellement      0.376582
executif          0.369085
fonctionnement    0.318713
representation    0.315582
cas               0.299169
Name: role, dtype: float32

cese


...

societe           1.000000
civile            0.976985
territoires       0.243575
representer       0.230222
representation    0.216963
representants     0.166118
nationale         0.112554
assemblee         0.103545
cese              0.100145
elus              0.099846
Name: societe, dtype: float32

nationale


nationale       1.000000
assemblee       0.482276
loi             0.196998
gouvernement    0.192733
lois            0.167607
senat           0.152119
deputes         0.139942
pouvoir         0.138587
territoires     0.124416
cese            0.116643
Name: nationale, dtype: float32

sert


sert               1.000000
chose              0.301724
social             0.160775
economique         0.154022
supprimer          0.147059
conseil            0.145426
cese               0.141823
environnemental    0.141414
senateurs          0.113381
senat              0.111889
Name: sert, dtype: float32

In [100]:
for i in df_qu_17_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_17[i].sort_values(ascending=False).head(10))

senat


senat            0.000180
chambre          0.000137
nationale        0.000132
represente       0.000127
assemblee        0.000126
supprime         0.000124
mode             0.000123
inutile          0.000119
collectivites    0.000118
suppression      0.000116
Name: senat, dtype: float32

role


role              0.000241
jouer             0.000212
actuel            0.000161
consultatif       0.000159
controle          0.000114
actuellement      0.000091
executif          0.000089
fonctionnement    0.000077
representation    0.000076
cas               0.000072
Name: role, dtype: float32

cese


...

societe           0.000831
civile            0.000811
territoires       0.000202
representer       0.000191
representation    0.000180
representants     0.000138
nationale         0.000093
assemblee         0.000086
cese              0.000083
elus              0.000083
Name: societe, dtype: float32

nationale


nationale       0.000866
assemblee       0.000418
loi             0.000171
gouvernement    0.000167
lois            0.000145
senat           0.000132
deputes         0.000121
pouvoir         0.000120
territoires     0.000108
cese            0.000101
Name: nationale, dtype: float32

sert


sert               0.000887
chose              0.000268
social             0.000143
economique         0.000137
supprimer          0.000130
conseil            0.000129
cese               0.000126
environnemental    0.000125
senateurs          0.000101
senat              0.000099
Name: sert, dtype: float32

In [101]:
gc.collect()

147

# Question 18 :  Faut-il les transformer ?

In [103]:
question_18 = "Faut-il les transformer ?"
qu_18,df_qu_18_one_hot,df_cov_18,df_confidence_18,df_lift_18 = compute_similarities(question_18, new_res)

In [104]:
for i in df_qu_18_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_18[i].sort_values(ascending=False).head(10))

oui


oui    22816
non        0
Name: oui, dtype: uint16

non


non    3381
oui       0
Name: non, dtype: uint16

In [105]:
for i in df_qu_18_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_18[i].sort_values(ascending=False).head(10))

oui


oui    1.0
non    0.0
Name: oui, dtype: float32

non


non    1.0
oui    0.0
Name: non, dtype: float32

In [106]:
for i in df_qu_18_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_18[i].sort_values(ascending=False).head(10))

oui


oui    0.000044
non         NaN
Name: oui, dtype: float32

non


non    0.000296
oui         NaN
Name: non, dtype: float32

In [107]:
gc.collect()

14

# Question 19 :  Si oui, comment ? (Senat)

In [108]:
question_19 = "Si oui, comment ?_Senat"
qu_19,df_qu_19_one_hot,df_cov_19,df_confidence_19,df_lift_19 = compute_similarities(question_19, new_res)

In [109]:
for i in df_qu_19_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_19[i].sort_values(ascending=False).head(10))

senat


senat          4950
cese           1422
supprimer      1415
assemblee      1122
suppression     836
conseil         826
nombre          751
elus            655
economique      654
citoyens        651
Name: senat, dtype: uint16

supprimer


supprimer     3575
senat         1415
cese           902
conseil        574
nombre         556
economique     496
senateurs      444
social         404
reduire        390
deputes        365
Name: supprimer, dtype: uint16

nombre


...

voir          1051
senat          152
reponse        152
sort             0
loi              0
direct           0
terrain          0
niveau           0
republique       0
tirage           0
Name: voir, dtype: uint16

role


role          1004
senat          463
cese           291
assemblee      205
conseil        187
citoyens       168
supprimer      166
nombre         165
senateurs      161
economique     139
Name: role, dtype: uint16

assemblees


assemblees    984
nombre        218
senat         206
citoyens      184
elus          184
assemblee     148
faut          132
reduire       124
cese          117
supprimer     116
Name: assemblees, dtype: uint16

In [110]:
for i in df_qu_19_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_19[i].sort_values(ascending=False).head(10))

senat


senat          1.000000
nationale      0.690992
chambre        0.610329
assemblee      0.597126
remplacer      0.582133
fusion         0.574545
fusionner      0.554479
territoires    0.541096
universel      0.520249
loi            0.516304
Name: senat, dtype: float32

supprimer


supprimer          1.000000
sert               0.432900
servent            0.377163
remplacer          0.360231
social             0.341505
economique         0.339959
conseil            0.324844
environnemental    0.309735
cese               0.302380
commissions        0.292929
Name: supprimer, dtype: float32

nombre


...

voir          1.000000
reponse       0.459215
senat         0.030707
sort          0.000000
loi           0.000000
direct        0.000000
terrain       0.000000
niveau        0.000000
republique    0.000000
tirage        0.000000
Name: voir, dtype: float32

role


role          1.000000
donner        0.165367
lois          0.159574
nationale     0.134550
pouvoir       0.121901
assemblee     0.109101
conseil       0.105829
social        0.104818
cese          0.097553
economique    0.095271
Name: role, dtype: float32

assemblees


assemblees    1.000000
faut          0.099623
citoyens      0.082511
elus          0.081996
assemblee     0.078765
reduire       0.070335
nombre        0.066728
senat         0.041616
cese          0.039222
supprimer     0.032448
Name: assemblees, dtype: float32

In [111]:
for i in df_qu_19_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_19[i].sort_values(ascending=False).head(10))

senat


senat          0.000202
nationale      0.000140
chambre        0.000123
assemblee      0.000121
remplacer      0.000118
fusion         0.000116
fusionner      0.000112
territoires    0.000109
universel      0.000105
loi            0.000104
Name: senat, dtype: float32

supprimer


supprimer          0.000280
sert               0.000121
servent            0.000106
remplacer          0.000101
social             0.000096
economique         0.000095
conseil            0.000091
environnemental    0.000087
cese               0.000085
commissions        0.000082
Name: supprimer, dtype: float32

nombre


...

voir               0.000951
reponse            0.000437
senat              0.000029
environnemental         NaN
citoyens                NaN
nombre                  NaN
cas                     NaN
mandats                 NaN
conseils                NaN
travail                 NaN
Name: voir, dtype: float32

role


role          0.000996
donner        0.000165
lois          0.000159
nationale     0.000134
pouvoir       0.000121
assemblee     0.000109
conseil       0.000105
social        0.000104
cese          0.000097
economique    0.000095
Name: role, dtype: float32

assemblees


assemblees    0.001016
faut          0.000101
citoyens      0.000084
elus          0.000083
assemblee     0.000080
reduire       0.000071
nombre        0.000068
senat         0.000042
cese          0.000040
supprimer     0.000033
Name: assemblees, dtype: float32

In [112]:
gc.collect()

147

# Question 20:   Que proposez-vous pour renforcer les principes de la laïcité dans le rapport entre l'Etat et les religions de notre pays ?

In [113]:
question_20 = "Que proposez-vous pour renforcer les principes de la laïcité dans le rapport entre l'Etat et les religions de notre pays ?"
qu_20,df_qu_20_one_hot,df_cov_20,df_confidence_20,df_lift_20 = compute_similarities(question_20, new_res)

In [114]:
for i in df_qu_20_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_20[i].sort_values(ascending=False).head(10))

laicite


laicite      6271
religions    1760
religion     1695
faire        1026
france        973
loi           936
faut          864
tout          850
tous          836
religieux     835
Name: laicite, dtype: uint16

religions


religions    6170
laicite      1760
religion     1482
loi           979
faire         867
france        859
religieux     813
pays          761
tout          760
faut          720
Name: religions, dtype: uint16

religion


...

lieux        1846
culte         710
religion      613
publics       592
religieux     537
laicite       514
religions     503
tous          432
cultes        378
interdire     363
Name: lieux, dtype: uint16

interdire


interdire    1825
religieux     625
religion      506
tout          473
religions     434
signes        399
lieux         363
tous          348
laicite       341
voile         320
Name: interdire, dtype: uint16

ecole


ecole        1786
laicite       604
religions     535
religion      435
religieux     363
faire         282
tous          259
tout          258
publique      230
education     223
Name: ecole, dtype: uint16

In [115]:
for i in df_qu_20_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_20[i].sort_values(ascending=False).head(10))

laicite


laicite      1.000000
principe     0.708839
principes    0.544796
veut         0.525526
sens         0.493939
faudrait     0.467262
probleme     0.464286
renforcer    0.455446
rappeler     0.448276
partout      0.447552
Name: laicite, dtype: float32

religions


religions        1.000000
representants    0.521523
respectent       0.512821
histoire         0.512162
adapter          0.509363
rapport          0.508600
maniere          0.483957
enseigner        0.479401
vis              0.472868
egalite          0.470687
Name: religions, dtype: float32

religion


...

lieux            1.000000
publics          0.556914
culte            0.468338
cultes           0.365217
construction     0.361486
publiques        0.332696
financement      0.245480
signe            0.230907
rue              0.229617
ostentatoires    0.224944
Name: lieux, dtype: float32

interdire


interdire        1.000000
port             0.314121
prieres          0.291429
etranger         0.283820
voile            0.279476
signes           0.268506
ostentatoires    0.262806
etrangers        0.245211
rue              0.242928
signe            0.230907
Name: interdire, dtype: float32

ecole


ecole           1.000000
enseigner       0.464419
civique         0.366120
cours           0.321127
enfants         0.315166
enseignement    0.298137
education       0.233753
publique        0.181531
laique          0.175971
histoire        0.171622
Name: ecole, dtype: float32

In [116]:
for i in df_qu_20_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_20[i].sort_values(ascending=False).head(10))

laicite


laicite      0.000159
principe     0.000113
principes    0.000087
veut         0.000084
sens         0.000079
faudrait     0.000075
probleme     0.000074
renforcer    0.000073
rappeler     0.000071
partout      0.000071
Name: laicite, dtype: float32

religions


religions        0.000162
representants    0.000085
respectent       0.000083
histoire         0.000083
adapter          0.000083
rapport          0.000082
maniere          0.000078
enseigner        0.000078
vis              0.000077
egalite          0.000076
Name: religions, dtype: float32

religion


...

lieux            0.000542
publics          0.000302
culte            0.000254
cultes           0.000198
construction     0.000196
publiques        0.000180
financement      0.000133
signe            0.000125
rue              0.000124
ostentatoires    0.000122
Name: lieux, dtype: float32

interdire


interdire        0.000548
port             0.000172
prieres          0.000160
etranger         0.000156
voile            0.000153
signes           0.000147
ostentatoires    0.000144
etrangers        0.000134
rue              0.000133
signe            0.000127
Name: interdire, dtype: float32

ecole


ecole           0.000560
enseigner       0.000260
civique         0.000205
cours           0.000180
enfants         0.000176
enseignement    0.000167
education       0.000131
publique        0.000102
laique          0.000099
histoire        0.000096
Name: ecole, dtype: float32

In [117]:
gc.collect()

147

# Question 21:  Comment garantir le respect par tous de la compréhension réciproque et des valeurs intangibles de la République ?

In [118]:
question_21 = "Comment garantir le respect par tous de la compréhension réciproque et des valeurs intangibles de la République ?"
qu_21,df_qu_21_one_hot,df_cov_21,df_confidence_21,df_lift_21 = compute_similarities(question_21, new_res)

In [119]:
for i in df_qu_21_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_21[i].sort_values(ascending=False).head(10))

education


education     6004
civique       2100
ecole         1683
valeurs        769
republique     638
cours          597
respect        539
tous           511
nationale      474
tout           414
Name: education, dtype: uint16

ecole


ecole          5382
education      1683
civique        1502
valeurs        1007
republique      792
instruction     600
respect         591
cours           551
primaire        532
faire           510
Name: ecole, dtype: uint16

valeurs


...

citoyens      1178
republique     350
valeurs        306
tous           299
education      219
respect        200
ecole          198
faire          165
tout           159
faut           152
Name: citoyens, dtype: uint16

france


france        1174
valeurs        373
republique     312
tous           245
francais       243
pays           218
faire          214
ecole          205
tout           193
education      187
Name: france, dtype: uint16

exemple


exemple       1134
valeurs        289
republique     252
elus           220
tous           185
ecole          174
education      166
respect        163
faire          153
tout           130
Name: exemple, dtype: uint16

In [120]:
for i in df_qu_21_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_21[i].sort_values(ascending=False).head(10))

education


education    1.000000
nationale    0.788686
civique      0.586265
citoyenne    0.528571
parents      0.498534
renforcer    0.497382
cours        0.445522
college      0.427015
long         0.425606
scolaire     0.419865
Name: education, dtype: float32

ecole


ecole            1.000000
primaire         0.621495
apprentissage    0.553299
enseigner        0.550305
lycee            0.543956
commence         0.543396
role             0.538321
apprendre        0.499192
parents          0.497067
college          0.488017
Name: ecole, dtype: float32

valeurs


...

citoyens      1.000000
elus          0.150794
droits        0.132641
tous          0.119029
exemple       0.110229
francais      0.104207
faut          0.100396
non           0.093041
republique    0.091265
tout          0.083861
Name: citoyens, dtype: float32

france


france       1.000000
histoire     0.298361
etrangers    0.251208
pays         0.237991
francais     0.232314
vivre        0.222037
francaise    0.189602
faut         0.106341
non          0.103206
tout         0.101793
Name: france, dtype: float32

exemple


exemple      1.000000
elus         0.249433
donner       0.234783
egalite      0.125153
citoyens     0.106112
faut         0.082563
respecter    0.074759
tous         0.073646
valeurs      0.068974
tout         0.068565
Name: exemple, dtype: float32

In [121]:
for i in df_qu_21_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_21[i].sort_values(ascending=False).head(10))

education


education    0.000167
nationale    0.000131
civique      0.000098
citoyenne    0.000088
parents      0.000083
renforcer    0.000083
cours        0.000074
college      0.000071
long         0.000071
scolaire     0.000070
Name: education, dtype: float32

ecole


ecole            0.000186
primaire         0.000115
apprentissage    0.000103
enseigner        0.000102
lycee            0.000101
commence         0.000101
role             0.000100
apprendre        0.000093
parents          0.000092
college          0.000091
Name: ecole, dtype: float32

valeurs


...

citoyens      0.000849
elus          0.000128
droits        0.000113
tous          0.000101
exemple       0.000094
francais      0.000088
faut          0.000085
non           0.000079
republique    0.000077
tout          0.000071
Name: citoyens, dtype: float32

france


france       0.000852
histoire     0.000254
etrangers    0.000214
pays         0.000203
francais     0.000198
vivre        0.000189
francaise    0.000162
faut         0.000091
non          0.000088
tout         0.000087
Name: france, dtype: float32

exemple


exemple      0.000882
elus         0.000220
donner       0.000207
egalite      0.000110
citoyens     0.000094
faut         0.000073
respecter    0.000066
tous         0.000065
valeurs      0.000061
tout         0.000060
Name: exemple, dtype: float32

In [122]:
gc.collect()

147

# Question 22:  Que faudrait-il faire aujourd'hui pour renforcer l'engagement citoyen dans la société ?

In [123]:
question_22 = "Que faudrait-il faire aujourd'hui pour renforcer l'engagement citoyen dans la société ?"
qu_22,df_qu_22_one_hot,df_cov_22,df_confidence_22,df_lift_22 = compute_similarities(question_22, new_res)

In [124]:
for i in df_qu_22_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_22[i].sort_values(ascending=False).head(10))

service


service        3604
civique        1559
obligatoire    1243
militaire      1178
mois            696
national        641
jeunes          609
tous            561
citoyen         540
place           309
Name: service, dtype: uint16

citoyens


citoyens      3524
faire          533
tous           412
elus           388
engagement     360
citoyen        356
societe        345
donner         323
vie            300
faut           290
Name: citoyens, dtype: uint16

citoyen


...

militaire      1200
service        1178
obligatoire     408
civique         308
mois            275
tous            183
remettre        181
jeunes          168
retablir        167
citoyen         143
Name: militaire, dtype: uint16

faut


faut          1192
citoyen        303
citoyens       290
engagement     234
faire          232
societe        181
tout           165
tous           163
service        140
vie            127
Name: faut, dtype: uint16

exemple


exemple         1148
citoyens         231
citoyen          200
faire            181
engagement       173
elus             157
service          147
tous             146
donner           145
associations     120
Name: exemple, dtype: uint16

In [125]:
for i in df_qu_22_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_22[i].sort_values(ascending=False).head(10))

service


service      1.000000
militaire    0.981667
civil        0.962382
6            0.861765
mois         0.811189
national     0.776029
civique      0.658918
an           0.621170
remettre     0.583524
bonne        0.574924
Name: service, dtype: float32

citoyens


citoyens      1.000000
sort          0.483709
decisions     0.364286
avis          0.362613
impliquer     0.336257
participer    0.324015
davantage     0.315476
permettre     0.309890
prendre       0.302932
engager       0.302135
Name: citoyens, dtype: float32

citoyen


...

militaire      1.000000
remettre       0.414188
6              0.402941
retablir       0.371111
service        0.326859
mois           0.320513
obligatoire    0.155963
jeunes         0.146853
civique        0.130178
place          0.122951
Name: militaire, dtype: float32

faut


faut          1.000000
societe       0.125173
tout          0.115952
engagement    0.108635
citoyen       0.101576
faire         0.099145
vie           0.096799
exemple       0.091463
tous          0.084852
citoyens      0.082293
Name: faut, dtype: float32

exemple


exemple       1.000000
politiques    0.142098
temps         0.122886
elus          0.113933
donner        0.097907
faut          0.088087
vie           0.083079
engagement    0.080316
societe       0.078147
faire         0.077350
Name: exemple, dtype: float32

In [126]:
for i in df_qu_22_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_22[i].sort_values(ascending=False).head(10))

service


service      0.000277
militaire    0.000272
civil        0.000267
6            0.000239
mois         0.000225
national     0.000215
civique      0.000183
an           0.000172
remettre     0.000162
bonne        0.000160
Name: service, dtype: float32

citoyens


citoyens      0.000284
sort          0.000137
decisions     0.000103
avis          0.000103
impliquer     0.000095
participer    0.000092
davantage     0.000090
permettre     0.000088
prendre       0.000086
engager       0.000086
Name: citoyens, dtype: float32

citoyen


...

militaire      0.000833
remettre       0.000345
6              0.000336
retablir       0.000309
service        0.000272
mois           0.000267
obligatoire    0.000130
jeunes         0.000122
civique        0.000108
place          0.000102
Name: militaire, dtype: float32

faut


faut          0.000839
societe       0.000105
tout          0.000097
engagement    0.000091
citoyen       0.000085
faire         0.000083
vie           0.000081
exemple       0.000077
tous          0.000071
citoyens      0.000069
Name: faut, dtype: float32

exemple


exemple       0.000871
politiques    0.000124
temps         0.000107
elus          0.000099
donner        0.000085
faut          0.000077
vie           0.000072
engagement    0.000070
societe       0.000068
faire         0.000067
Name: exemple, dtype: float32

In [127]:
gc.collect()

147

# Question 23:   Quels sont les comportements civiques qu'il faut promouvoir dans notre vie quotidienne ou collective ?

In [128]:
question_23 = "Quels sont les comportements civiques qu'il faut promouvoir dans notre vie quotidienne ou collective ?"
qu_23,df_qu_23_one_hot,df_cov_23,df_confidence_23,df_lift_23 = compute_similarities(question_23, new_res)

In [129]:
for i in df_qu_23_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_23[i].sort_values(ascending=False).head(10))

respect


respect          11683
politesse         1346
tolerance          998
solidarite         823
environnement      794
tous               704
lois               696
entraide           675
biens              657
institutions       605
Name: respect, dtype: uint16

politesse


politesse     1762
respect       1346
proprete       157
vivre          156
entraide       156
tolerance      145
courtoisie     130
savoir         125
regles         116
solidarite     105
Name: politesse, dtype: uint16

solidarite


...

faut             889
respect          252
promouvoir       213
faire            155
comportements    125
tout             116
tous             116
vie              107
societe          101
associatif         0
Name: faut, dtype: uint16

ecole


ecole            868
respect          387
civique          176
education        154
parents          111
droits             0
tolerance          0
france             0
publics            0
participation      0
Name: ecole, dtype: uint16

citoyens


citoyens         845
respect          285
tous             126
comportement       0
collectif          0
tolerance          0
france             0
publics            0
participation      0
empathie           0
Name: citoyens, dtype: uint16

In [130]:
for i in df_qu_23_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_23[i].sort_values(ascending=False).head(10))

respect


respect          1.000000
biens            0.874834
institutions     0.854520
difference       0.854406
differences      0.831579
autorite         0.815789
lois             0.782902
regles           0.769728
politesse        0.763905
environnement    0.733826
Name: respect, dtype: float32

politesse


politesse     1.000000
courtoisie    0.430464
savoir        0.280269
proprete      0.252412
vivre         0.175084
regles        0.150065
respect       0.115210
entraide      0.107216
tolerance     0.103498
solidarite    0.061368
Name: politesse, dtype: float32

solidarite


...

faut             1.000000
promouvoir       0.292985
societe          0.157813
faire            0.142332
comportements    0.137817
tout             0.093775
vie              0.084921
tous             0.070388
respect          0.021570
associatif       0.000000
Name: faut, dtype: float32

ecole


ecole            1.000000
parents          0.301630
civique          0.229765
education        0.171875
respect          0.033125
droits           0.000000
tolerance        0.000000
france           0.000000
publics          0.000000
participation    0.000000
Name: ecole, dtype: float32

citoyens


citoyens         1.000000
tous             0.076456
respect          0.024394
comportement     0.000000
collectif        0.000000
tolerance        0.000000
france           0.000000
publics          0.000000
participation    0.000000
empathie         0.000000
Name: citoyens, dtype: float32

In [131]:
for i in df_qu_23_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_23[i].sort_values(ascending=False).head(10))

respect


respect          0.000086
biens            0.000075
institutions     0.000073
difference       0.000073
differences      0.000071
autorite         0.000070
lois             0.000067
regles           0.000066
politesse        0.000065
environnement    0.000063
Name: respect, dtype: float32

politesse


politesse     0.000568
courtoisie    0.000244
savoir        0.000159
proprete      0.000143
vivre         0.000099
regles        0.000085
respect       0.000065
entraide      0.000061
tolerance     0.000059
solidarite    0.000035
Name: politesse, dtype: float32

solidarite


...

faut             0.001125
promouvoir       0.000330
societe          0.000178
faire            0.000160
comportements    0.000155
tout             0.000105
vie              0.000096
tous             0.000079
respect          0.000024
benevolat             NaN
Name: faut, dtype: float32

ecole


ecole            0.001152
parents          0.000348
civique          0.000265
education        0.000198
respect          0.000038
benevolat             NaN
bienveillance         NaN
dechets               NaN
citoyens              NaN
travail               NaN
Name: ecole, dtype: float32

citoyens


citoyens         0.001183
tous             0.000090
respect          0.000029
benevolat             NaN
bienveillance         NaN
dechets               NaN
travail               NaN
laicite               NaN
commun                NaN
respecter             NaN
Name: citoyens, dtype: float32

In [132]:
gc.collect()

147

# Question 24:  Que faudrait-il faire pour favoriser le développement de ces comportements civiques et par quels engagements concrets chacun peut-il y participer ?

In [133]:
question_24 = "Que faudrait-il faire pour favoriser le développement de ces comportements civiques et par quels engagements concrets chacun peut-il y participer ?"
qu_24,df_qu_24_one_hot,df_cov_24,df_confidence_24,df_lift_24 = compute_similarities(question_24, new_res)

In [134]:
for i in df_qu_24_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_24[i].sort_values(ascending=False).head(10))

education


education    3849
ecole         975
civique       887
parents       585
enfants       429
tout          372
nationale     315
faire         290
service       269
age           253
Name: education, dtype: uint16

ecole


ecole          3335
education       975
civique         742
parents         564
enfants         381
faire           349
tout            318
respect         309
service         278
instruction     271
Name: ecole, dtype: uint16

civique


...

jeunes           872
service          256
civique          212
education        192
ecole            186
faire            140
tous             137
comportements      0
tout               0
france             0
Name: jeunes, dtype: uint16

donner


donner           866
exemple          247
faire            136
temps            129
education        122
moyens           111
ecole            108
publics            0
niveau             0
participation      0
Name: donner, dtype: uint16

non


non              831
respect          199
faire            139
ecole            134
education        134
tous             108
comportements    104
tout             104
valoriser          0
france             0
Name: non, dtype: uint16

In [135]:
for i in df_qu_24_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_24[i].sort_values(ascending=False).head(10))

education


education    1.000000
nationale    0.748219
parents      0.449654
renforcer    0.411552
cours        0.391225
scolaire     0.389034
famille      0.384106
civique      0.370200
enfants      0.365106
role         0.363083
Name: education, dtype: float32

ecole


ecole          1.000000
primaire       0.641667
enseigner      0.550943
morale         0.546075
instruction    0.508443
famille        0.500000
parents        0.433513
role           0.427992
cours          0.422303
apprendre      0.421505
Name: ecole, dtype: float32

civique


...

jeunes           1.000000
service          0.143498
tous             0.093964
civique          0.088481
faire            0.072614
ecole            0.055772
education        0.049883
tout             0.000000
comportements    0.000000
france           0.000000
Name: jeunes, dtype: float32

donner


donner           1.000000
moyens           0.335347
temps            0.194570
exemple          0.143856
faire            0.070539
ecole            0.032384
education        0.031697
publics          0.000000
niveau           0.000000
participation    0.000000
Name: donner, dtype: float32

non


non              1.000000
respect          0.175795
comportements    0.081954
tous             0.074074
tout             0.072524
faire            0.072095
ecole            0.040180
education        0.034814
valoriser        0.000000
france           0.000000
Name: non, dtype: float32

In [136]:
for i in df_qu_24_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_24[i].sort_values(ascending=False).head(10))

education


education    0.000260
nationale    0.000194
parents      0.000117
renforcer    0.000107
cours        0.000102
scolaire     0.000101
famille      0.000100
civique      0.000096
enfants      0.000095
role         0.000094
Name: education, dtype: float32

ecole


ecole          0.000300
primaire       0.000192
enseigner      0.000165
morale         0.000164
instruction    0.000152
famille        0.000150
parents        0.000130
role           0.000128
cours          0.000127
apprendre      0.000126
Name: ecole, dtype: float32

civique


...

jeunes       0.001147
service      0.000165
tous         0.000108
civique      0.000101
faire        0.000083
ecole        0.000064
education    0.000057
benevolat         NaN
eduquer           NaN
sanctions         NaN
Name: jeunes, dtype: float32

donner


donner       0.001155
moyens       0.000387
temps        0.000225
exemple      0.000166
faire        0.000081
ecole        0.000037
education    0.000037
benevolat         NaN
eduquer           NaN
sanctions         NaN
Name: donner, dtype: float32

non


non              0.001203
respect          0.000212
comportements    0.000099
tous             0.000089
tout             0.000087
faire            0.000087
ecole            0.000048
education        0.000042
benevolat             NaN
eduquer               NaN
Name: non, dtype: float32

In [137]:
gc.collect()

147

# Question 25:  Que faudrait-il faire pour valoriser l'engagement citoyen dans les parcours de vie, dans les relations avec l'administration et les pouvoirs publics ?

In [138]:
question_25 = "Que faudrait-il faire pour valoriser l'engagement citoyen dans les parcours de vie, dans les relations avec l'administration et les pouvoirs publics ?"
qu_25,df_qu_25_one_hot,df_cov_25,df_confidence_25,df_lift_25 = compute_similarities(question_25, new_res)

In [139]:
for i in df_qu_25_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_25[i].sort_values(ascending=False).head(10))

citoyens


citoyens           1948
administration      370
publics             290
faire               243
pouvoirs            236
citoyen             207
vie                 183
administrations     178
engagement          178
tous                168
Name: citoyens, dtype: uint16

citoyen


citoyen           1895
engagement         838
administration     339
vie                226
service            214
publics            212
citoyens           207
faire              199
valoriser          190
pouvoirs           176
Name: citoyen, dtype: uint16

administration


...

temps         702
travail       157
engagement    136
citoyen       134
retraite      121
citoyens      101
donner        101
internet        0
pouvoirs        0
faudrait        0
Name: temps, dtype: uint16

retraite


retraite      657
points        161
engagement    161
compte        133
temps         121
citoyen       103
internet        0
pouvoirs        0
faudrait        0
sais            0
Name: retraite, dtype: uint16

civique


civique        652
service        380
education      134
obligatoire    118
tout             0
pouvoirs         0
faudrait         0
sais             0
engagement       0
internet         0
Name: civique, dtype: uint16

In [140]:
for i in df_qu_25_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_25[i].sort_values(ascending=False).head(10))

citoyens


citoyens           1.000000
pouvoirs           0.290283
elus               0.263033
ecoute             0.248227
publics            0.245971
pouvoir            0.240185
administrations    0.223618
administration     0.216248
non                0.214403
tous               0.207921
Name: citoyens, dtype: float32

citoyen


citoyen           1.000000
engagement        0.555335
parcours          0.285714
valoriser         0.227545
vie               0.226453
pouvoirs          0.216482
faut              0.214653
administration    0.198130
non               0.196399
compte            0.193603
Name: citoyen, dtype: float32

administration


...

temps         1.000000
travail       0.263866
retraite      0.184170
donner        0.139696
engagement    0.090126
citoyen       0.070712
citoyens      0.051848
internet      0.000000
pouvoirs      0.000000
faudrait      0.000000
Name: temps, dtype: float32

retraite


retraite      1.000000
points        0.501558
compte        0.223906
temps         0.172365
engagement    0.106693
citoyen       0.054354
internet      0.000000
pouvoirs      0.000000
faudrait      0.000000
sais          0.000000
Name: retraite, dtype: float32

civique


civique        1.000000
obligatoire    0.336182
service        0.306205
education      0.233043
tout           0.000000
pouvoirs       0.000000
faudrait       0.000000
sais           0.000000
engagement     0.000000
internet       0.000000
Name: civique, dtype: float32

In [141]:
for i in df_qu_25_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_25[i].sort_values(ascending=False).head(10))

citoyens


citoyens           0.000513
pouvoirs           0.000149
elus               0.000135
ecoute             0.000127
publics            0.000126
pouvoir            0.000123
administrations    0.000115
administration     0.000111
non                0.000110
tous               0.000107
Name: citoyens, dtype: float32

citoyen


citoyen           0.000528
engagement        0.000293
parcours          0.000151
valoriser         0.000120
vie               0.000120
pouvoirs          0.000114
faut              0.000113
administration    0.000105
non               0.000104
compte            0.000102
Name: citoyen, dtype: float32

administration


...

temps              0.001425
travail            0.000376
retraite           0.000262
donner             0.000199
engagement         0.000128
citoyen            0.000101
citoyens           0.000074
benevolat               NaN
administrations         NaN
administration          NaN
Name: temps, dtype: float32

retraite


retraite           0.001522
points             0.000763
compte             0.000341
temps              0.000262
engagement         0.000162
citoyen            0.000083
benevolat               NaN
citoyens                NaN
administrations         NaN
administration          NaN
Name: retraite, dtype: float32

civique


civique            0.001534
obligatoire        0.000516
service            0.000470
education          0.000357
benevolat               NaN
citoyens                NaN
administrations         NaN
administration          NaN
emploi                  NaN
cas                     NaN
Name: civique, dtype: float32

In [142]:
gc.collect()

147

# Question 26:  Quelles sont les incivilités les plus pénibles dans la vie quotidienne et que faudrait-il faire pour lutter contre ces incivilités ?

In [143]:
question_26 = "Quelles sont les incivilités les plus pénibles dans la vie quotidienne et que faudrait-il faire pour lutter contre ces incivilités ?"
qu_26,df_qu_26_one_hot,df_cov_26,df_confidence_26,df_lift_26 = compute_similarities(question_26, new_res)

In [144]:
for i in df_qu_26_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_26[i].sort_values(ascending=False).head(10))

respect


respect        6666
non            3904
manque         2012
route           854
code            710
education       700
regles          690
publics         623
biens           620
incivilites     480
Name: respect, dtype: uint16

non


non            4943
respect        3904
route           774
code            659
regles          599
publics         500
biens           472
incivilites     427
faire           399
dechets         377
Name: non, dtype: uint16

incivilites


...

transports     1227
commun          532
respect         331
publics         293
non             283
rue             224
incivilites     198
route           155
dechets         141
insultes        137
Name: transports, dtype: uint16

public


public          1147
espace           452
respect          396
non              357
degradation      182
degradations     179
dechets          162
incivilites      123
publics          117
tout             105
Name: public, dtype: uint16

agressions


agressions      1142
verbales         435
physiques        269
respect          213
vols             210
non              186
insultes         180
degradations     142
femmes           121
publics          120
Name: agressions, dtype: uint16

In [145]:
for i in df_qu_26_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_26[i].sort_values(ascending=False).head(10))

respect


respect          1.000000
non              0.789804
manque           0.746845
regles           0.729387
code             0.726714
environnement    0.612688
route            0.563696
valeurs          0.537162
agees            0.519573
politesse        0.512777
Name: respect, dtype: float32

non


non              1.000000
zones            0.833828
code             0.674514
regles           0.633192
respect          0.585659
droit            0.576087
route            0.510891
environnement    0.479132
pietons          0.436842
vitesse          0.431981
Name: non, dtype: float32

incivilites


...

transports       1.000000
commun           0.690013
fraude           0.260181
publics          0.156100
trottoirs        0.155488
rue              0.154483
femmes           0.108108
comportements    0.107803
route            0.102310
public           0.088056
Name: transports, dtype: float32

public


public          1.000000
espace          0.788831
degradation     0.180915
degradations    0.110290
dechets         0.090909
transports      0.082315
non             0.072223
tout            0.064457
publics         0.062334
respect         0.059406
Name: public, dtype: float32

agressions


agressions      1.000000
verbales        0.613540
physiques       0.555785
vols            0.264484
femmes          0.116795
insultes        0.097986
degradations    0.087492
rue             0.075172
faut            0.070280
publics         0.063932
Name: agressions, dtype: float32

In [146]:
for i in df_qu_26_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_26[i].sort_values(ascending=False).head(10))

respect


respect          0.000150
non              0.000118
manque           0.000112
regles           0.000109
code             0.000109
environnement    0.000092
route            0.000085
valeurs          0.000081
agees            0.000078
politesse        0.000077
Name: respect, dtype: float32

non


non              0.000202
zones            0.000169
code             0.000136
regles           0.000128
respect          0.000118
droit            0.000117
route            0.000103
environnement    0.000097
pietons          0.000088
vitesse          0.000087
Name: non, dtype: float32

incivilites


...

transports       0.000815
commun           0.000562
fraude           0.000212
publics          0.000127
trottoirs        0.000127
rue              0.000126
femmes           0.000088
comportements    0.000088
route            0.000083
public           0.000072
Name: transports, dtype: float32

public


public          0.000872
espace          0.000688
degradation     0.000158
degradations    0.000096
dechets         0.000079
transports      0.000072
non             0.000063
tout            0.000056
publics         0.000054
respect         0.000052
Name: public, dtype: float32

agressions


agressions      0.000876
verbales        0.000537
physiques       0.000487
vols            0.000232
femmes          0.000102
insultes        0.000086
degradations    0.000077
rue             0.000066
faut            0.000062
publics         0.000056
Name: agressions, dtype: float32

In [147]:
gc.collect()

133

# Question 27: Que peuvent et doivent faire les pouvoirs publics pour répondre aux incivilités ?

In [148]:
question_27 = "Que peuvent et doivent faire les pouvoirs publics pour répondre aux incivilités ?"
qu_27,df_qu_27_one_hot,df_cov_27,df_confidence_27,df_lift_27 = compute_similarities(question_27, new_res)

In [149]:
for i in df_qu_27_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_27[i].sort_values(ascending=False).head(10))

faire


faire          3351
respecter       683
loi             668
appliquer       621
lois            457
payer           359
incivilites     311
tout            260
justice         256
police          248
Name: faire, dtype: uint16

sanctionner


sanctionner    2664
eduquer         343
incivilites     253
faire           236
travaux         199
interet         196
general         159
severement      143
parents         128
amendes         123
Name: sanctionner, dtype: uint16

education


...

faut           1056
faire           231
incivilites     169
sanctionner     118
education       109
tout            107
justice         104
appliquer         0
tig               0
tolerance         0
Name: faut, dtype: uint16

punir


punir            1043
severement        151
faire             131
eduquer           119
incivilites       101
laxisme             0
incivilite          0
delinquants         0
parents             0
comportements       0
Name: punir, dtype: uint16

tout


tout             1011
faire             260
education         137
loi               108
faut              107
mettre              0
voir                0
delinquants         0
parents             0
comportements       0
Name: tout, dtype: uint16

In [150]:
for i in df_qu_27_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_27[i].sort_values(ascending=False).head(10))

faire


faire        1.000000
respecter    0.827879
payer        0.741736
reparer      0.418251
lois         0.383389
loi          0.361668
appliquer    0.332441
regles       0.295890
tout         0.257171
travail      0.247851
Name: faire, dtype: float32

sanctionner


sanctionner    1.000000
severement     0.362944
eduquer        0.282770
incivilites    0.153987
general        0.145604
interet        0.141926
parents        0.139282
travaux        0.134097
faut           0.111742
exemple        0.095319
Name: sanctionner, dtype: float32

education


...

faut           1.000000
tout           0.105836
incivilites    0.102861
justice        0.074713
faire          0.068935
education      0.049840
sanctionner    0.044294
appliquer      0.000000
tig            0.000000
tolerance      0.000000
Name: faut, dtype: float32

punir


punir            1.000000
severement       0.383249
eduquer          0.098104
incivilites      0.061473
faire            0.039093
laxisme          0.000000
incivilite       0.000000
delinquants      0.000000
parents          0.000000
comportements    0.000000
Name: punir, dtype: float32

tout


tout             1.000000
faut             0.101326
faire            0.077589
education        0.062643
loi              0.058473
mettre           0.000000
voir             0.000000
delinquants      0.000000
parents          0.000000
comportements    0.000000
Name: tout, dtype: float32

In [151]:
for i in df_qu_27_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_27[i].sort_values(ascending=False).head(10))

faire


faire        0.000298
respecter    0.000247
payer        0.000221
reparer      0.000125
lois         0.000114
loi          0.000108
appliquer    0.000099
regles       0.000088
tout         0.000077
travail      0.000074
Name: faire, dtype: float32

sanctionner


sanctionner    0.000375
severement     0.000136
eduquer        0.000106
incivilites    0.000058
general        0.000055
interet        0.000053
parents        0.000052
travaux        0.000050
faut           0.000042
exemple        0.000036
Name: sanctionner, dtype: float32

education


...

faut           0.000947
tout           0.000100
incivilites    0.000097
justice        0.000071
faire          0.000065
education      0.000047
sanctionner    0.000042
eduquer             NaN
sanctions           NaN
forces              NaN
Name: faut, dtype: float32

punir


punir          0.000959
severement     0.000367
eduquer        0.000094
incivilites    0.000059
faire          0.000037
sanctions           NaN
forces              NaN
amende              NaN
citoyens            NaN
peines              NaN
Name: punir, dtype: float32

tout


tout         0.000989
faut         0.000100
faire        0.000077
education    0.000062
loi          0.000058
eduquer           NaN
sanctions         NaN
forces            NaN
amende            NaN
citoyens          NaN
Name: tout, dtype: float32

In [152]:
gc.collect()

147

# Question 28: Quel pourrait être le rôle de chacun pour faire reculer les incivilités dans la société ?

In [153]:
question_28 = "Quel pourrait être le rôle de chacun pour faire reculer les incivilités dans la société ?"
qu_28,df_qu_28_one_hot,df_cov_28,df_confidence_28,df_lift_28 = compute_similarities(question_28, new_res)

In [154]:
for i in df_qu_28_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_28[i].sort_values(ascending=False).head(10))

faire


faire          2422
incivilites     333
respecter       206
police          180
enfants         174
incivilite      172
tout            164
faut            155
exemple         154
education       153
Name: faire, dtype: uint16

education


education      1930
enfants         436
parents         375
ecole           232
role            184
faire           153
respect         150
tout            145
civique         115
incivilites     108
Name: education, dtype: uint16

exemple


...

donner          681
exemple         477
comportement      0
sais              0
commencer         0
difficile         0
civique           0
ordre             0
signaler          0
loi               0
Name: donner, dtype: uint16

citoyen


citoyen          660
faire            147
comportement       0
voir               0
apprendre          0
parents            0
comportements      0
tout               0
prendre            0
incivilite         0
Name: citoyen, dtype: uint16

citoyens


citoyens         652
faire            147
incivilites      112
prendre            0
apprendre          0
parents            0
comportements      0
tout               0
voir               0
loi                0
Name: citoyens, dtype: uint16

In [155]:
for i in df_qu_28_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_28[i].sort_values(ascending=False).head(10))

faire


faire          1.000000
justice        0.260163
police         0.252454
peur           0.229287
faut           0.226608
citoyens       0.225460
citoyen        0.222727
incivilites    0.219947
respecter      0.218684
incivilite     0.216898
Name: faire, dtype: float32

education


education      1.000000
civique        0.458167
parents        0.376128
ecole          0.371200
enfants        0.268473
role           0.182359
tout           0.168801
respect        0.133215
incivilites    0.071334
faire          0.063171
Name: education, dtype: float32

exemple


...

donner          1.000000
exemple         0.269492
comportement    0.000000
sais            0.000000
commencer       0.000000
difficile       0.000000
civique         0.000000
ordre           0.000000
signaler        0.000000
loi             0.000000
Name: donner, dtype: float32

citoyen


citoyen          1.000000
faire            0.060694
comportement     0.000000
voir             0.000000
apprendre        0.000000
parents          0.000000
comportements    0.000000
tout             0.000000
prendre          0.000000
incivilite       0.000000
Name: citoyen, dtype: float32

citoyens


citoyens         1.000000
incivilites      0.073976
faire            0.060694
prendre          0.000000
apprendre        0.000000
parents          0.000000
comportements    0.000000
tout             0.000000
voir             0.000000
loi              0.000000
Name: citoyens, dtype: float32

In [156]:
for i in df_qu_28_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_28[i].sort_values(ascending=False).head(10))

faire


faire          0.000413
justice        0.000107
police         0.000104
peur           0.000095
faut           0.000094
citoyens       0.000093
citoyen        0.000092
incivilites    0.000091
respecter      0.000090
incivilite     0.000090
Name: faire, dtype: float32

education


education      0.000518
civique        0.000237
parents        0.000195
ecole          0.000192
enfants        0.000139
role           0.000094
tout           0.000087
respect        0.000069
incivilites    0.000037
faire          0.000033
Name: education, dtype: float32

exemple


...

donner       0.001468
exemple      0.000396
eduquer           NaN
forces            NaN
citoyens          NaN
cas               NaN
travail           NaN
pouvoir           NaN
respecter         NaN
regles            NaN
Name: donner, dtype: float32

citoyen


citoyen      0.001515
faire        0.000092
eduquer           NaN
forces            NaN
citoyens          NaN
cas               NaN
travail           NaN
pouvoir           NaN
respecter         NaN
regles            NaN
Name: citoyen, dtype: float32

citoyens


citoyens       0.001534
incivilites    0.000113
faire          0.000093
eduquer             NaN
forces              NaN
cas                 NaN
travail             NaN
pouvoir             NaN
respecter           NaN
regles              NaN
Name: citoyens, dtype: float32

In [157]:
gc.collect()

147

# Question 29: Quelles sont les discriminations les plus répandues dont vous êtes témoin ou victime ?

In [158]:
question_29 = "Quelles sont les discriminations les plus répandues dont vous êtes témoin ou victime ?"
qu_29,df_qu_29_one_hot,df_cov_29,df_confidence_29,df_lift_29 = compute_similarities(question_29, new_res)

In [159]:
for i in df_qu_29_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_29[i].sort_values(ascending=False).head(10))

racisme


racisme           4301
homophobie         930
sexisme            827
anti               620
blanc              404
discrimination     249
femmes             226
francais           205
antisemitisme      199
temoin             134
Name: racisme, dtype: uint16

discrimination


discrimination     3029
femmes              491
racisme             249
sociale             221
femme               219
raciale             207
embauche            202
temoin              190
discriminations     186
homme               183
Name: discrimination, dtype: uint16

femmes


...

age               547
discrimination    156
couleur             0
ans                 0
jeunes              0
non                 0
antisemitisme       0
femmes              0
harcelement         0
anti                0
Name: age, dtype: uint16

origine


origine            544
discrimination     143
discriminations    124
couleur              0
france               0
jeunes               0
non                  0
antisemitisme        0
femmes               0
harcelement          0
Name: origine, dtype: uint16

raciales


raciales           540
discriminations    240
ans                  0
jeunes               0
non                  0
antisemitisme        0
femmes               0
harcelement          0
anti                 0
discrimination       0
Name: raciales, dtype: uint16

In [160]:
for i in df_qu_29_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_29[i].sort_values(ascending=False).head(10))

racisme


racisme          1.000000
anti             0.672451
homophobie       0.622907
antisemitisme    0.580175
blanc            0.579627
sexisme          0.559162
blancs           0.359431
francais         0.267275
temoin           0.200000
non              0.118074
Name: racisme, dtype: float32

discrimination


discrimination    1.000000
embauche          0.556474
raciale           0.552000
sociale           0.423372
victime           0.346000
emploi            0.318681
homme             0.285937
age               0.285192
temoin            0.283582
travail           0.275362
Name: discrimination, dtype: float32

femmes


...

age               1.000000
discrimination    0.051502
couleur           0.000000
ans               0.000000
jeunes            0.000000
non               0.000000
antisemitisme     0.000000
femmes            0.000000
harcelement       0.000000
anti              0.000000
Name: age, dtype: float32

origine


origine            1.000000
discriminations    0.059472
discrimination     0.047210
couleur            0.000000
france             0.000000
jeunes             0.000000
non                0.000000
antisemitisme      0.000000
femmes             0.000000
harcelement        0.000000
Name: origine, dtype: float32

raciales


raciales           1.000000
discriminations    0.115108
ans                0.000000
jeunes             0.000000
non                0.000000
antisemitisme      0.000000
femmes             0.000000
harcelement        0.000000
anti               0.000000
discrimination     0.000000
Name: raciales, dtype: float32

In [161]:
for i in df_qu_29_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_29[i].sort_values(ascending=False).head(10))

racisme


racisme          0.000233
anti             0.000156
homophobie       0.000145
antisemitisme    0.000135
blanc            0.000135
sexisme          0.000130
blancs           0.000084
francais         0.000062
temoin           0.000047
non              0.000027
Name: racisme, dtype: float32

discrimination


discrimination    0.000330
embauche          0.000184
raciale           0.000182
sociale           0.000140
victime           0.000114
emploi            0.000105
homme             0.000094
age               0.000094
temoin            0.000094
travail           0.000091
Name: discrimination, dtype: float32

femmes


...

age               0.001828
discrimination    0.000094
emploi                 NaN
travail                NaN
victime                NaN
vis                    NaN
transports             NaN
handicap               NaN
acces                  NaN
pauvres                NaN
Name: age, dtype: float32

origine


origine            0.001838
discriminations    0.000109
discrimination     0.000087
emploi                  NaN
travail                 NaN
victime                 NaN
vis                     NaN
transports              NaN
handicap                NaN
acces                   NaN
Name: origine, dtype: float32

raciales


raciales           0.001852
discriminations    0.000213
emploi                  NaN
travail                 NaN
victime                 NaN
vis                     NaN
transports              NaN
handicap                NaN
acces                   NaN
pauvres                 NaN
Name: raciales, dtype: float32

In [162]:
gc.collect()

147

# Question 30: Que faudrait-il faire pour lutter contre ces discriminations et construire une société plus solidaire et plus tolérante ?

In [163]:
question_30 = "Que faudrait-il faire pour lutter contre ces discriminations et construire une société plus solidaire et plus tolérante ?"
qu_30,df_qu_30_one_hot,df_cov_30,df_confidence_30,df_lift_30 = compute_similarities(question_30, new_res)

In [164]:
for i in df_qu_30_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_30[i].sort_values(ascending=False).head(10))

education


education    3339
ecole         466
tout          273
civique       269
parents       231
enfants       214
tous          199
faire         192
age           189
jeune         175
Name: education, dtype: uint16

faire


faire        2054
tout          256
tous          249
respecter     236
faut          224
societe       213
lois          213
education     192
ecole         189
appliquer     165
Name: faire, dtype: uint16

ecole


...

sanctionner    641
eduquer        144
non              0
sens             0
tolerance        0
france           0
voir             0
tolerante        0
prendre          0
niveau           0
Name: sanctionner, dtype: uint16

lois


lois           637
faire          213
appliquer      194
respecter      122
immigration      0
tolerance        0
france           0
voir             0
tolerante        0
prendre          0
Name: lois, dtype: uint16

vie


vie          637
tous         131
societe      127
faire        109
tout         102
appliquer      0
tolerance      0
france         0
voir           0
tolerante      0
Name: vie, dtype: uint16

In [165]:
for i in df_qu_30_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_30[i].sort_values(ascending=False).head(10))

education


education    1.000000
nationale    0.650000
civique      0.542339
parents      0.422303
jeune        0.335893
age          0.315526
enfants      0.295580
ecole        0.283111
jeunes       0.261603
respect      0.203681
Name: education, dtype: float32

faire


faire        1.000000
respecter    0.476768
lois         0.334380
appliquer    0.316699
loi          0.239669
monde        0.199634
arreter      0.197995
travail      0.197597
faut         0.191945
france       0.186992
Name: faire, dtype: float32

ecole


...

sanctionner    1.000000
eduquer        0.108108
non            0.000000
sens           0.000000
tolerance      0.000000
france         0.000000
voir           0.000000
tolerante      0.000000
prendre        0.000000
niveau         0.000000
Name: sanctionner, dtype: float32

lois


lois           1.000000
appliquer      0.372361
respecter      0.246465
faire          0.103700
immigration    0.000000
tolerance      0.000000
france         0.000000
voir           0.000000
tolerante      0.000000
prendre        0.000000
Name: lois, dtype: float32

vie


vie          1.000000
societe      0.101357
tous         0.087979
tout         0.068872
faire        0.053067
appliquer    0.000000
tolerance    0.000000
france       0.000000
voir         0.000000
tolerante    0.000000
Name: vie, dtype: float32

In [166]:
for i in df_qu_30_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_30[i].sort_values(ascending=False).head(10))

education


education    0.000299
nationale    0.000195
civique      0.000162
parents      0.000126
jeune        0.000101
age          0.000094
enfants      0.000089
ecole        0.000085
jeunes       0.000078
respect      0.000061
Name: education, dtype: float32

faire


faire        0.000487
respecter    0.000232
lois         0.000163
appliquer    0.000154
loi          0.000117
monde        0.000097
arreter      0.000096
travail      0.000096
faut         0.000093
france       0.000091
Name: faire, dtype: float32

ecole


...

sanctionner    0.001560
eduquer        0.000169
sanctions           NaN
citoyens            NaN
cas                 NaN
travail             NaN
pouvoir             NaN
respecter           NaN
montrer             NaN
creer               NaN
Name: sanctionner, dtype: float32

lois


lois         0.001570
appliquer    0.000585
respecter    0.000387
faire        0.000163
eduquer           NaN
sanctions         NaN
citoyens          NaN
cas               NaN
travail           NaN
pouvoir           NaN
Name: lois, dtype: float32

vie


vie          0.001570
societe      0.000159
tous         0.000138
tout         0.000108
faire        0.000083
eduquer           NaN
sanctions         NaN
citoyens          NaN
cas               NaN
travail           NaN
Name: vie, dtype: float32

In [167]:
gc.collect()

147

# Question 31:  Pensez-vous qu'il faille instaurer des contreparties aux différentes allocations de solidarité ?

In [168]:
question_31 = "Pensez-vous qu'il faille instaurer des contreparties aux différentes allocations de solidarité ?"
qu_31,df_qu_31_one_hot,df_cov_31,df_confidence_31,df_lift_31 = compute_similarities(question_31, new_res)

In [169]:
for i in df_qu_31_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_31[i].sort_values(ascending=False).head(10))

oui


oui    21329
non        0
Name: oui, dtype: uint16

non


non    7795
oui       0
Name: non, dtype: uint16

In [170]:
for i in df_qu_31_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_31[i].sort_values(ascending=False).head(10))

oui


oui    1.0
non    0.0
Name: oui, dtype: float32

non


non    1.0
oui    0.0
Name: non, dtype: float32

In [171]:
for i in df_qu_31_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_31[i].sort_values(ascending=False).head(10))

oui


oui    0.000047
non         NaN
Name: oui, dtype: float32

non


non    0.000128
oui         NaN
Name: non, dtype: float32

In [172]:
gc.collect()

14

# Question 32: Si oui, lesquelles ?

In [173]:
question_32 = "Si oui, lesquelles ?"
qu_32,df_qu_32_one_hot,df_cov_32,df_confidence_32,df_lift_32 = compute_similarities(question_32, new_res)

In [174]:
for i in df_qu_32_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_32[i].sort_values(ascending=False).head(10))

travail


travail         4328
interet         1336
general         1081
heures           585
allocations      534
rsa              428
aide             393
faire            382
tout             361
collectivite     358
Name: travail, dtype: uint16

interet


interet         3213
general         2331
travaux         1595
travail         1336
heures           283
collectif        281
allocations      264
aide             260
associations     259
faire            241
Name: interet, dtype: uint16

allocations


...

solidarite      1152
allocations      482
contrepartie     219
travail          209
allocation       209
faire            169
faut             156
societe          146
non              145
aide             136
Name: solidarite, dtype: uint16

respect


respect        1146
regles          269
lois            261
allocations     200
non             189
republique      179
enfants         123
cas             117
travail         116
societe         102
Name: respect, dtype: uint16

tout


tout            1130
travail          361
allocations      245
faire            199
salaire          190
monde            177
faut             165
aides            163
aide             153
contrepartie     137
Name: tout, dtype: uint16

In [175]:
for i in df_qu_32_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_32[i].sort_values(ascending=False).head(10))

travail


travail          1.000000
salaire          0.604061
collectif        0.514360
monde            0.444805
general          0.440147
heures           0.432373
interet          0.415811
collectivites    0.388535
utilite          0.372294
semaine          0.369963
Name: travail, dtype: float32

interet


interet      1.000000
general      0.949104
collectif    0.733681
travaux      0.584463
public       0.427632
generaux     0.415270
travail      0.308688
chomeurs     0.247166
agees        0.245115
semaine      0.214286
Name: interet, dtype: float32

allocations


...

solidarite      1.000000
contrepartie    0.182348
allocation      0.165348
allocations     0.158188
non             0.143281
faut            0.138175
societe         0.129318
faire           0.110820
tout            0.102655
cas             0.097723
Name: solidarite, dtype: float32

respect


respect        1.000000
lois           0.683246
regles         0.603139
republique     0.521866
non            0.186759
cas            0.111006
societe        0.090345
enfants        0.090110
allocations    0.065638
travail        0.026802
Name: respect, dtype: float32

tout


tout            1.000000
monde           0.574675
salaire         0.482233
droit           0.212679
faut            0.146147
faire           0.130492
contrepartie    0.114072
societe         0.112489
allocation      0.103639
aides           0.100866
Name: tout, dtype: float32

In [176]:
for i in df_qu_32_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_32[i].sort_values(ascending=False).head(10))

travail


travail          0.000231
salaire          0.000140
collectif        0.000119
monde            0.000103
general          0.000102
heures           0.000100
interet          0.000096
collectivites    0.000090
utilite          0.000086
semaine          0.000085
Name: travail, dtype: float32

interet


interet      0.000311
general      0.000295
collectif    0.000228
travaux      0.000182
public       0.000133
generaux     0.000129
travail      0.000096
chomeurs     0.000077
agees        0.000076
semaine      0.000067
Name: interet, dtype: float32

allocations


...

solidarite      0.000868
contrepartie    0.000158
allocation      0.000144
allocations     0.000137
non             0.000124
faut            0.000120
societe         0.000112
faire           0.000096
tout            0.000089
cas             0.000085
Name: solidarite, dtype: float32

respect


respect        0.000873
lois           0.000596
regles         0.000526
republique     0.000455
non            0.000163
cas            0.000097
societe        0.000079
enfants        0.000079
allocations    0.000057
travail        0.000023
Name: respect, dtype: float32

tout


tout            0.000885
monde           0.000509
salaire         0.000427
droit           0.000188
faut            0.000129
faire           0.000115
contrepartie    0.000101
societe         0.000100
allocation      0.000092
aides           0.000089
Name: tout, dtype: float32

In [177]:
gc.collect()

147

# Question 33:  Que pensez-vous de la situation de l'immigration en France aujourd'hui et de la politique migratoire ? Quelles sont, selon vous, les critères à mettre en place pour définir la politique migratoire ?

In [178]:
question_33 = "Que pensez-vous de la situation de l'immigration en France aujourd'hui et de la politique migratoire ? Quelles sont, selon vous, les critères à mettre en place pour définir la politique migratoire ?"
qu_33,df_qu_33_one_hot,df_cov_33,df_confidence_33,df_lift_33 = compute_similarities(question_33, new_res)

In [179]:
for i in df_qu_33_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_33[i].sort_values(ascending=False).head(10))

immigration


immigration    8656
pays           2825
france         2683
faut           2070
politique      1603
choisie        1334
francais       1221
asile          1053
faire          1020
non            1005
Name: immigration, dtype: uint16

pays


pays           7765
immigration    2825
france         2628
faut           1994
politique      1642
migrants       1540
accueillir     1530
origine        1267
faire          1228
accueil        1124
Name: pays, dtype: uint16

france


...

quotas         2105
immigration     674
pays            587
faut            442
france          434
besoins         364
politique       353
mettre          351
place           325
fonction        292
Name: quotas, dtype: uint16

non


non            2036
immigration    1005
pays            833
france          739
faut            523
politique       501
francais        440
migrants        381
faire           355
droit           322
Name: non, dtype: uint16

monde


monde          1872
accueillir      930
france          867
pays            783
misere          764
tout            702
immigration     592
faut            539
politique       328
faire           328
Name: monde, dtype: uint16

In [180]:
for i in df_qu_33_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_33[i].sort_values(ascending=False).head(10))

immigration


immigration    1.000000
clandestine    0.950877
choisie        0.916838
illegale       0.900000
controlee      0.769437
stopper        0.702083
importante     0.684358
economique     0.616646
chance         0.604585
reduire        0.581152
Name: immigration, dtype: float32

pays


pays             1.000000
origine          0.869595
developpement    0.820833
developper       0.747405
guerre           0.684250
europeens        0.678010
fuient           0.661130
aider            0.659538
danger           0.621138
guerres          0.618182
Name: pays, dtype: float32

france


...

quotas          1.000000
instaurer       0.522078
fonction        0.245791
mettre          0.223282
definir         0.215553
besoins         0.205766
place           0.198292
canada          0.177458
familial        0.152399
regroupement    0.140551
Name: quotas, dtype: float32

non


non           1.000000
oui           0.342183
emplois       0.300948
respect       0.192819
culture       0.168874
choisie       0.167010
enfants       0.166667
ans           0.161245
republique    0.159398
aides         0.158594
Name: non, dtype: float32

monde


monde         1.000000
misere        0.710037
pouvons       0.401099
tout          0.302195
accueillir    0.240559
part          0.201087
prendre       0.181240
moyens        0.180612
terre         0.166441
vivre         0.158981
Name: monde, dtype: float32

In [181]:
for i in df_qu_33_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_33[i].sort_values(ascending=False).head(10))

immigration


immigration    0.000116
clandestine    0.000110
choisie        0.000106
illegale       0.000104
controlee      0.000089
stopper        0.000081
importante     0.000079
economique     0.000071
chance         0.000070
reduire        0.000067
Name: immigration, dtype: float32

pays


pays             0.000129
origine          0.000112
developpement    0.000106
developper       0.000096
guerre           0.000088
europeens        0.000087
fuient           0.000085
aider            0.000085
danger           0.000080
guerres          0.000080
Name: pays, dtype: float32

france


...

quotas          0.000475
instaurer       0.000248
fonction        0.000117
mettre          0.000106
definir         0.000102
besoins         0.000098
place           0.000094
canada          0.000084
familial        0.000072
regroupement    0.000067
Name: quotas, dtype: float32

non


non           0.000491
oui           0.000168
emplois       0.000148
respect       0.000095
culture       0.000083
choisie       0.000082
enfants       0.000082
ans           0.000079
republique    0.000078
aides         0.000078
Name: non, dtype: float32

monde


monde         0.000534
misere        0.000379
pouvons       0.000214
tout          0.000161
accueillir    0.000129
part          0.000107
prendre       0.000097
moyens        0.000096
terre         0.000089
vivre         0.000085
Name: monde, dtype: float32

In [182]:
gc.collect()

112

# Question 34:  En matière d'immigration, une fois nos obligations d'asile remplies, souhaitez-vous que nous puissions nous fixer des objectifs annuels définis par le Parlement ?

In [183]:
question_34 = "En matière d'immigration, une fois nos obligations d'asile remplies, souhaitez-vous que nous puissions nous fixer des objectifs annuels définis par le Parlement ?"
qu_34,df_qu_34_one_hot,df_cov_34,df_confidence_34,df_lift_34 = compute_similarities(question_34, new_res)

In [184]:
for i in df_qu_34_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_34[i].sort_values(ascending=False).head(10))

oui


oui            12589
pays             700
immigration      606
besoins          592
quotas           581
fonction         570
objectifs        569
faut             567
france           360
asile            333
Name: oui, dtype: uint16

non


non            5426
pays            420
immigration     396
faut            377
objectifs       343
asile           317
quotas          303
france          299
oui             270
faire           201
Name: non, dtype: uint16

pays


...

migrants      777
pays          246
oui           203
non           162
faut          152
france        151
asile         135
accueillir    129
quotas        114
objectifs     103
Name: migrants, dtype: uint16

fixer


fixer          766
objectifs      393
faut           202
quotas         194
oui            193
pays           157
immigration    138
annuels        136
asile          126
parlement      114
Name: fixer, dtype: uint16

tout


tout           763
pays           189
oui            180
monde          174
faut           163
non            157
asile          148
accueillir     135
france         134
immigration    119
Name: tout, dtype: uint16

In [185]:
for i in df_qu_34_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_34[i].sort_values(ascending=False).head(10))

oui


oui            1.000000
fonction       0.551257
canada         0.549080
besoins        0.513889
choisie        0.432836
emploi         0.414634
main           0.407115
integration    0.403377
compte         0.402827
oeuvre         0.400697
Name: oui, dtype: float32

non


non          1.000000
cas          0.308889
francais     0.251928
droit        0.223776
tous         0.221580
faire        0.211802
france       0.211756
politique    0.210210
migrants     0.208494
tout         0.205767
Name: non, dtype: float32

pays


...

migrants      1.000000
accueillir    0.133540
pays          0.106957
france        0.106941
faut          0.080508
asile         0.073250
quotas        0.064153
objectifs     0.049710
non           0.029856
oui           0.016125
Name: migrants, dtype: float32

fixer


fixer          1.000000
annuels        0.234483
objectifs      0.189672
parlement      0.126667
quotas         0.109173
faut           0.106992
asile          0.068367
pays           0.068261
immigration    0.066219
non            0.020641
Name: fixer, dtype: float32

tout


tout           1.000000
monde          0.333333
accueillir     0.139752
france         0.094901
faut           0.086335
pays           0.082174
asile          0.080304
immigration    0.057102
objectifs      0.050193
non            0.028935
Name: tout, dtype: float32

In [186]:
for i in df_qu_34_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_34[i].sort_values(ascending=False).head(10))

oui


oui            0.000079
fonction       0.000044
canada         0.000044
besoins        0.000041
choisie        0.000034
emploi         0.000033
main           0.000032
integration    0.000032
compte         0.000032
oeuvre         0.000032
Name: oui, dtype: float32

non


non          0.000184
cas          0.000057
francais     0.000046
droit        0.000041
tous         0.000041
faire        0.000039
france       0.000039
politique    0.000039
migrants     0.000038
tout         0.000038
Name: non, dtype: float32

pays


...

migrants      0.001287
accueillir    0.000172
pays          0.000138
france        0.000138
faut          0.000104
asile         0.000094
quotas        0.000083
objectifs     0.000064
non           0.000038
oui           0.000021
Name: migrants, dtype: float32

fixer


fixer          0.001305
annuels        0.000306
objectifs      0.000248
parlement      0.000165
quotas         0.000143
faut           0.000140
asile          0.000089
pays           0.000089
immigration    0.000086
non            0.000027
Name: fixer, dtype: float32

tout


tout           0.001311
monde          0.000437
accueillir     0.000183
france         0.000124
faut           0.000113
pays           0.000108
asile          0.000105
immigration    0.000075
objectifs      0.000066
non            0.000038
Name: tout, dtype: float32

In [187]:
gc.collect()

147

# Question 35: Que proposez-vous afin de répondre à ce défi qui va durer ?

In [188]:
question_35 = "Que proposez-vous afin de répondre à ce défi qui va durer ?"
qu_35,df_qu_35_one_hot,df_cov_35,df_confidence_35,df_lift_35 = compute_similarities(question_35, new_res)

In [189]:
for i in df_qu_35_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_35[i].sort_values(ascending=False).head(10))

pays


pays             9062
aider            2297
origine          1756
developpement    1532
migrants         1317
immigration      1284
faut             1188
france           1179
faire            1173
aide             1085
Name: pays, dtype: uint16

immigration


immigration    3157
pays           1284
france          545
faut            434
politique       425
faire           382
choisie         378
economique      305
aider           292
origine         286
Name: immigration, dtype: uint16

aider


...

aides            1307
pays              703
sociales          278
france            244
migrants          214
immigration       208
francais          182
faire             175
aider             174
developpement     170
Name: aides, dtype: uint16

tous


tous           1301
pays            628
france          269
faire           228
faut            224
migrants        212
immigration     208
francais        162
aider           156
politique       149
Name: tous, dtype: uint16

tout


tout           1293
pays            559
france          278
faire           260
monde           255
faut            254
immigration     202
francais        195
migrants        182
accueillir      164
Name: tout, dtype: uint16

In [190]:
for i in df_qu_35_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_35[i].sort_values(ascending=False).head(10))

pays


pays             1.000000
origine          0.949189
concernes        0.942350
emigration       0.888620
depart           0.866667
pauvres          0.854123
developpement    0.832156
dirigeants       0.825095
aider            0.813673
developper       0.811594
Name: pays, dtype: float32

immigration


immigration     1.000000
choisie         0.945000
stopper         0.466899
limiter         0.333841
economique      0.311542
besoins         0.254045
familial        0.252525
regroupement    0.248408
fonction        0.246316
lutter          0.230189
Name: immigration, dtype: float32

aider


...

aides            1.000000
sociales         0.679707
supprimer        0.284289
donner           0.138070
droit            0.129611
francais         0.113892
non              0.105670
developpement    0.092341
france           0.091660
immigres         0.087777
Name: aides, dtype: float32

tous


tous        1.000000
asile       0.130712
monde       0.128510
droit       0.115653
europe      0.110351
immigres    0.109926
francais    0.101377
france      0.101052
tout        0.100541
faut        0.096344
Name: tous, dtype: float32

tout


tout          1.000000
monde         0.275378
territoire    0.188000
vie           0.126794
francais      0.122028
droit         0.111665
accueillir    0.111036
faut          0.109247
travail       0.108883
france        0.104433
Name: tout, dtype: float32

In [191]:
for i in df_qu_35_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_35[i].sort_values(ascending=False).head(10))

pays


pays             0.000110
origine          0.000105
concernes        0.000104
emigration       0.000098
depart           0.000096
pauvres          0.000094
developpement    0.000092
dirigeants       0.000091
aider            0.000090
developper       0.000090
Name: pays, dtype: float32

immigration


immigration     0.000317
choisie         0.000299
stopper         0.000148
limiter         0.000106
economique      0.000099
besoins         0.000080
familial        0.000080
regroupement    0.000079
fonction        0.000078
lutter          0.000073
Name: immigration, dtype: float32

aider


...

aides            0.000765
sociales         0.000520
supprimer        0.000218
donner           0.000106
droit            0.000099
francais         0.000087
non              0.000081
developpement    0.000071
france           0.000070
immigres         0.000067
Name: aides, dtype: float32

tous


tous        0.000769
asile       0.000100
monde       0.000099
droit       0.000089
europe      0.000085
immigres    0.000084
francais    0.000078
france      0.000078
tout        0.000077
faut        0.000074
Name: tous, dtype: float32

tout


tout          0.000773
monde         0.000213
territoire    0.000145
vie           0.000098
francais      0.000094
droit         0.000086
accueillir    0.000086
faut          0.000084
travail       0.000084
france        0.000081
Name: tout, dtype: float32

In [192]:
gc.collect()

147

# Question 36:  Quelles sont, selon vous, les modalités d'intégration les plus efficaces et les plus justes à mettre en place aujourd'hui dans la société ?

In [193]:
question_36 = "Quelles sont, selon vous, les modalités d'intégration les plus efficaces et les plus justes à mettre en place aujourd'hui dans la société ?"
qu_36,df_qu_36_one_hot,df_cov_36,df_confidence_36,df_lift_36 = compute_similarities(question_36, new_res)

In [194]:
for i in df_qu_36_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_36[i].sort_values(ascending=False).head(10))

langue


langue           6317
apprentissage    2207
francaise        2105
travail          1149
apprendre        1126
pays             1002
integration       977
valeurs           816
formation         775
culture           750
Name: langue, dtype: uint16

francais


francais         5547
apprendre        1274
apprentissage    1059
france            977
travail           858
integration       816
cours             783
pays              732
langue            661
francaise         642
Name: francais, dtype: uint16

travail


...

respect          1513
langue            538
lois              350
valeurs           332
francais          330
pays              324
laicite           294
francaise         290
republique        284
apprentissage     283
Name: respect, dtype: uint16

culture


culture          1498
langue            750
francaise         491
francais          424
pays              340
apprentissage     316
integration       309
france            255
histoire          229
apprendre         221
Name: culture, dtype: uint16

tout


tout           1434
francais        439
langue          340
integration     308
pays            277
france          271
faut            233
travail         232
faire           226
francaise       215
Name: tout, dtype: uint16

In [195]:
for i in df_qu_36_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_36[i].sort_values(ascending=False).head(10))

langue


langue           1.000000
maitrise         0.791809
francaise        0.708516
apprentissage    0.686043
connaissance     0.671315
institutions     0.557927
principes        0.517787
histoire         0.511526
culture          0.500668
metier           0.486957
Name: langue, dtype: float32

francais


francais      1.000000
cours         0.633495
sol           0.554455
examen        0.553571
apprendre     0.536647
savoir        0.534535
obligation    0.505187
connaitre     0.461883
niveau        0.421053
femmes        0.416031
Name: francais, dtype: float32

travail


...

respect       1.000000
lois          0.269438
laicite       0.268248
regles        0.201824
republique    0.181586
valeurs       0.169734
non           0.165031
coutumes      0.157692
culture       0.116155
pays          0.112578
Name: respect, dtype: float32

culture


culture      1.000000
histoire     0.251372
francaise    0.165264
langue       0.118727
pays         0.118138
respect      0.115003
cours        0.109223
valeurs      0.107873
faut         0.104280
france       0.102041
Name: culture, dtype: float32

tout


tout          1.000000
monde         0.490029
territoire    0.233967
faut          0.140446
droits        0.123529
faire         0.122096
non           0.114184
donner        0.110423
societe       0.109244
france        0.108443
Name: tout, dtype: float32

In [196]:
for i in df_qu_36_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_36[i].sort_values(ascending=False).head(10))

langue


langue           0.000158
maitrise         0.000125
francaise        0.000112
apprentissage    0.000109
connaissance     0.000106
institutions     0.000088
principes        0.000082
histoire         0.000081
culture          0.000079
metier           0.000077
Name: langue, dtype: float32

francais


francais      0.000180
cours         0.000114
sol           0.000100
examen        0.000100
apprendre     0.000097
savoir        0.000096
obligation    0.000091
connaitre     0.000083
niveau        0.000076
femmes        0.000075
Name: francais, dtype: float32

travail


...

respect       0.000661
lois          0.000178
laicite       0.000177
regles        0.000133
republique    0.000120
valeurs       0.000112
non           0.000109
coutumes      0.000104
culture       0.000077
pays          0.000074
Name: respect, dtype: float32

culture


culture      0.000668
histoire     0.000168
francaise    0.000110
langue       0.000079
pays         0.000079
respect      0.000077
cours        0.000073
valeurs      0.000072
faut         0.000070
france       0.000068
Name: culture, dtype: float32

tout


tout          0.000697
monde         0.000342
territoire    0.000163
faut          0.000098
droits        0.000086
faire         0.000085
non           0.000080
donner        0.000077
societe       0.000076
france        0.000076
Name: tout, dtype: float32

In [ ]:
gc.collect()

147

# Question 37:   Y a-t-il d'autres points sur la démocratie et la citoyenneté sur lesquels vous souhaiteriez vous exprimer ?

In [ ]:
question_37 = "Y a-t-il d'autres points sur la démocratie et la citoyenneté sur lesquels vous souhaiteriez vous exprimer ?"
qu_37,df_qu_37_one_hot,df_cov_37,df_confidence_37,df_lift_37 = compute_similarities(question_37, new_res)

In [ ]:
for i in df_qu_37_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_37[i].sort_values(ascending=False).head(10))

In [ ]:
for i in df_qu_37_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_37[i].sort_values(ascending=False).head(10))

In [ ]:
for i in df_qu_37_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_37[i].sort_values(ascending=False).head(10))

In [ ]:
gc.collect()